In [ ]:
# Load the plot setup for thesis quality plots
from mpl_latex_settings import create_latex_setup
config = "/Users/tmenne/git/thesis/inc/mpl_latex_config.json"
latex_setup = create_latex_setup(config, interactive=True)

In [ ]:
from __future__ import division, print_function
import os
import json
import gzip
import healpy as hp
from glob import glob
import dill

import numpy as np
import matplotlib.pyplot as plt
import scipy.interpolate as sci
import scipy.stats as scs
import scipy.optimize as sco
from scipy.ndimage.filters import gaussian_filter
from matplotlib import gridspec
from matplotlib.colors import LogNorm, LinearSegmentedColormap
from astropy.time import Time as astrotime

import tdepps.utils as tdu
from tdepps.utils.stats import fit_chi2_cdf, weighted_cdf
from tdepps.utils.stats import sigma2prob, prob2sigma, delta_chi2
from tdepps.utils.stats import percentile_nzeros, cdf_nzeros
arr2str = tdu.arr2str

from mypyscripts.general import round_mantissa
import mypyscripts.plots as mplt
from mypyscripts.plots import (skymap, plot_healpy_map, mollview,
                               render_healpymap)

from _loader import loader as LOADER
from _paths import PATHS_ORIG as PATHS  # Use the original 22 HESE sources
from _plots import idx2rowcol

# Make some globals
SECINDAY = 24. * 60. * 60.
RNDGEN = np.random.RandomState(42439462)
loader = LOADER(PATHS, verb=False)

print("Paths are:\n", PATHS)

def save_fig(fig, path, **save_args):
    path = os.path.abspath(path)
    dirname = os.path.dirname(path)
    if not os.path.isdir(dirname):
        os.makedirs(dirname)   
    fig.savefig(path, **save_args)
    
def key2label(key, short=False):
    """ Makes 'IC86, 2012--2014' from 'IC86_2012-2104' """
    if short:  # Abbreviate 20YY -> 'YY
        return ", ".join(map(lambda s: s.replace("-", "--").replace("20", "'"),
                             key.split("_")))
    return ", ".join(map(lambda s: s.replace("-", "--"), key.split("_")))

def trunc_cmap(cmap, arr):
    """
    Adapted from: https://stackoverflow.com/questions/18926031
    Makes a new colormap with original values selected from arr in [0, 1].
    """
    vmin, vmax = np.amin(arr), np.amax(arr)
    new_cmap = LinearSegmentedColormap.from_list(
        'trunc({}_{:.2f}_{:.2f})'.format(cmap.name, vmin, vmax), cmap(arr))
    return new_cmap

# Chapter: Datasets

## Count number of events originally in the data sets

In [ ]:
exp = np.load("/Users/tmenne/Downloads/hese_transient_stacking_data/skylab_data/IC79b_exp.npy")
print("IC79: ", len(exp))
exp = np.load("/Users/tmenne/Downloads/hese_transient_stacking_data/skylab_data/IC86_exp.npy")
print("IC86, 2011: ", len(exp))

exp = np.load("/Users/tmenne/Downloads/hese_transient_stacking_data/skylab_data/IC86-2012_exp_v2.npy")
print("IC86, 2012: ", len(exp))
exp = np.load("/Users/tmenne/Downloads/hese_transient_stacking_data/skylab_data/IC86-2013_exp_v2.npy")
print("IC86, 2013: ", len(exp))
exp = np.load("/Users/tmenne/Downloads/hese_transient_stacking_data/skylab_data/IC86-2014_exp_v2.npy")
print("IC86, 2014: ", len(exp))

exp = np.load("/Users/tmenne/Downloads/hese_transient_stacking_data/skylab_data/SplineMPEmax.MuEx.IC86-2015.npy")
print("IC86, 2015: ", len(exp))

## Print out source info for table

In [ ]:
src_list = loader.source_list_loader("all")

src_decs = []
src_ras = []
src_mjds = []
for name, sl in sorted(src_list.items()):
    src_decs = np.concatenate([src_decs, [sli["dec"] for sli in sl]])
    src_ras = np.concatenate([src_ras, [sli["ra"] for sli in sl]])
    src_mjds = np.concatenate([src_mjds, [sli["mjd"] for sli in sl]])

In [ ]:
print(arr2str(np.rad2deg(src_decs), fmt="{:.2f}"))
print("")
print(arr2str(np.rad2deg(src_ras), fmt="{:.2f}"))
print("")
print(arr2str(src_mjds, fmt="{:.2f}"))

## Show HESE / no HESE in MC

The region high energy region from the northern sky should be investigated.
In true variables it looks like statistics, but in proxies it blows up enormuously.
Should the main effect come from the souther high energy sky?

In [ ]:
def remove_hese_from_mc(mc, heseids):
    """
    Mask all values in ``mc`` that have the same run and event ID combination
    as in ``heseids``.

    Parameters
    ----------
    mc : record-array
        MC data, needs names ``'Run', 'Event'``.
    heseids : dict or record-array
        Needs names / keys ``'run_id', 'event_id``.

    Returns
    -------
    is_hese_like : array-like, shape (len(mc),)
        Mask: ``True`` for each event in ``mc`` that is HESE like.
    """
    # Make combined IDs to easily match against HESE IDs with `np.isin`
    factor_mc = 10**np.ceil(np.log10(np.amax(mc["Event"])))
    _evids = np.atleast_1d(heseids["event_id"])
    factor_hese = 10**np.ceil(np.log10(np.amax(_evids)))
    factor = max(factor_mc, factor_hese)

    combined_mcids = (factor * mc["Run"] + mc["Event"]).astype(int)
    assert np.all(combined_mcids > factor)  # Is int overflow a thing here?

    _runids = np.atleast_1d(heseids["run_id"])
    combined_heseids = (factor * _runids + _evids).astype(int)
    assert np.all(combined_heseids > factor)

    # Check which MC event is tagged as HESE like
    is_hese_like = np.isin(combined_mcids, combined_heseids)
    print("  Found {} / {} HESE like events in MC".format(np.sum(is_hese_like),
                                                          len(mc)))
    return is_hese_like

def _get_weights(ow, trueE):
    """ Common model to weight to event rate """ 
    lt = 365. * SECINDAY
    return ow * tdu.power_law_flux(trueE, E0=1e5,
                                   phi0=1.01e-18, gamma=2.19) * lt

name2skylab = {
    "IC79" : "IC79b_corrected_MC.npy",
    "IC86_2011" : "IC86_corrected_MC.npy",
    "IC86_2012-2014" : "IC86-2012_corrected_MC_v2.npy",
    "IC86_2015" : "SplineMPEmax.MuEx.MC.npy",
}
name2idx = {
    "IC79" : "IC79.json.gz",
    "IC86_2011" : "IC86_2011.json.gz",
    "IC86_2012-2014" : "IC86_2012-2015.json.gz",
    "IC86_2015" : "IC86_2012-2015.json.gz",    
}

In [ ]:
proxy = False  # Switch False | True to see plots in trueE, trueDec, vs. proxies
if proxy:
    _sindec = lambda mc: np.sin(mc["dec"])
    _logE = lambda mc: mc["logE"]
    _logE_label = r"$\log_{10}(E_\mathrm{proxy} / \mathrm{GeV})$"
    _sindec_label = r"$\sin(\delta_\mathrm{proxy})$"
    _info = "_proxy"
else:
    _sindec = lambda mc: np.sin(mc["trueDec"])
    _logE = lambda mc: np.log10(mc["trueE"])
    _logE_label = r"$\log_{10}(E_\nu / \mathrm{GeV})$"
    _sindec_label = r"$\sin(\delta_\nu)$"
    _info = ""

for sample_name in name2skylab.keys()[:]:
    print("# {}".format(sample_name))
    # Load full skylab data
    _mc = np.load(os.path.join(PATHS.skylab_data, name2skylab[sample_name]))

    # Filter HESE events
    _path = os.path.join(PATHS.local, "check_hese_mc_ids",
                         name2idx[sample_name])
    heseids = json.load(gzip.open(_path))
    is_hese_mask = remove_hese_from_mc(_mc, heseids)
    _mc_hese_only = _mc[is_hese_mask]
    # Get weights
    w_all = _get_weights(_mc["ow"], _mc["trueE"])
    w_hese_only = _get_weights(_mc_hese_only["ow"], _mc_hese_only["trueE"])

    # Plot filtered out HESE events
    try:
        fs = latex_setup.make_figsize(scale=0.95, ratio=2.5)
    except:
        fs = (10, 4)
    fig, (axl, axr) = plt.subplots(1, 2, sharey=True, figsize=fs)
    
    # Make histograms for contourf
    bins = [np.linspace(-1, 1, 40), np.linspace(2, 9, 40)]
            # np.linspace(np.amin(_logE(_mc)), np.amax(_logE(_mc)), 40)]
    h, _, _ = np.histogram2d(_sindec(_mc), _logE(_mc), normed=False,
                             bins=bins, weights=w_all)
    h_nh, _, _ = np.histogram2d(_sindec(_mc_hese_only), _logE(_mc_hese_only),
                                bins=bins, weights=w_hese_only, normed=False)
    vmin = 10**np.floor(np.amin(np.log10(h[h>0])))
    vmax = 10**np.ceil(np.amax(np.log10(h[h>0])))

    # Plot whole MC left, weights normed to 365 days
    h_pad = np.pad(h.T, mode="edge", pad_width=[(0, 1), (0, 1)])
    levels = np.logspace(-4, 0, 11)
    img = axl.contourf(bins[0], bins[1], h_pad, levels,
                       cmap="Greys", norm=LogNorm())
    cbar = fig.colorbar(img, ax=axl, ticks=levels[::2])
    cbar.ax.set_yticklabels([])
    cbar.ax.set_yticks(levels[::2])
#     cbar.ax.set_yticklabels(["$10^{{{:.0f}}}$".format(li)
#                              for li in np.log10(levels[::2])])
    
    # Right plot only cut-out HESE, weights normed to 365 days
    h_nh_pad = np.pad(h_nh.T, mode="edge", pad_width=[(0, 1), (0, 1)])
    img = axr.contourf(bins[0], bins[1], h_nh_pad, levels,
                       cmap="Greys", norm=LogNorm())
    cbar = fig.colorbar(img, ax=axr, ticks=levels[::2])
    cbar.ax.set_yticklabels(["$10^{{{:.0f}}}$".format(li)
                             for li in np.log10(levels[::2])])
    cbar.set_label("Differenial no. of events")

#     axl.set_title("No. events for whole MC: {:.2f}".format(np.sum(w_all)))
#     axr.set_title("No. events for whole HESE-like MC: " +
#                   "{:.2f}".format(np.sum(w_hese_only)))
    for ax in (axl, axr):
        ax.set_xlabel(_sindec_label)
        ax.set_ylim(bins[1][0], bins[1][-1])
        ax.grid(ls=":")
    axl.set_ylabel(_logE_label)
    
    fig.tight_layout(w_pad=-0.2)
    
    save_fig(fig, "plots/datasets/mc_no_hese_{}.pdf".format(sample_name),
             bbox_inches="tight")
#     save_fig(fig, "plots/sample/mc_no_hese_{}.pgf".format(sample_name),
#              bbox_inches="tight")

    plt.show()

Old plots compared ratios

In [ ]:
fig, ax = plt.subplots(1, 1)

ratio_pad = np.pad(ratio, pad_width=[(0, 1), (0, 1)], mode="edge")
levels = np.linspace(0.6, 1., 9)
img = ax.contourf(bins[0], bins[1], ratio_pad.T, levels,
                  cmap="Greys_r", extend="min")
fig.colorbar(img, ax=ax)

plt.show()


# Or compare to pcolormesh
fig, ax = plt.subplots(1, 1)

img = ax.pcolormesh(bins[0], bins[1], ratio.T, cmap="Greys_r", vmin=0.75)
fig.colorbar(img, ax=ax)

plt.show()

## Source positions

In [ ]:
src_list = loader.source_list_loader("all")
src_maps = {k: loader.source_map_loader(sl) for k, sl in src_list.items()}

In [ ]:
try:
    fs = latex_setup.make_figsize(ratio=2.5, scale=1.)
except:
    fs = (6, 4)

One version with the reco maps

In [ ]:
src_map_sum = np.sum(
    [np.sum(srcm, axis=0) for srcm in src_maps.values()], axis=0)

skym = skymap()
fig, ax = skym.figure(
    None, None, proj="mollweide", grid=True, gal_plane=True,
    tex=True, grid_move_to_back=False, gal_plane_label=None,
    figsize=fs)
# Don't overdo with dpi and levels when using contourf -> large files!
dpi = 72
fig.set_dpi(val=dpi)  # To set the pcolormesh xx, yy dimension

# Old plot with pcolormesh and rasterized
# plot_healpy_map(ax=ax, m=src_map_sum, cmap="Greys", rasterize=True)
levels = np.round(scs.chi2.sf(np.arange(1., 3. + 1., 1.)[::-1]**2, df=2) *
                  np.amax(src_map_sum))

cmap = trunc_cmap(plt.cm.Greys, np.r_[0., np.linspace(0.6, 1.0, 3)])
plot_healpy_map(ax=ax, m=src_map_sum, cmap=cmap, extend="both",
                levels=levels, draw_contour=True)
# plt.legend(loc="upper right")

save_fig(plt.gcf(), "plots/datasets/hese_events_reco_landscape_skymap.pdf",
         bbox_inches="tight")

plt.show()

And one version with the sources per sample

In [ ]:
skym = skymap()
fig, ax = skym.figure(
    None, None, proj="mollweide", grid=True, gal_plane=True,
    tex=True, grid_move_to_back=False, figsize=fs, gal_plane_label=None)

marker = ["x", "d", "+", "s"]
ms = [5, 3, 5, 3]
colors = 4 * ["k"]
for i, (name, srcs) in enumerate(sorted(src_list.items())):
    src_ra = np.array([src["ra"] for src in srcs])
    src_dec = np.array([src["dec"] for src in srcs])
    
    short = False
    if name == "IC86_2012-2014":
        short = True
    
    x, y = skym.EquCoordsToMapCoords(dec=src_dec, ra=src_ra)   
    ax.plot(x, y, color=colors[i], marker=marker[i],
            ls="", ms=ms[i], label=key2label(name, short=short))
    
leg = ax.legend(
    loc="upper left", ncol=1,
    # bbox_to_anchor=(0.5, 0.925),
    # bbox_transform=plt.gcf().transFigure,
    prop={"size": latex_setup.latex_settings["legend.fontsize"] - 3},
    framealpha=1.)

save_fig(plt.gcf(), "plots/datasets//hese_events_per_sample_skymap.pdf",
         bbox_inches="tight")
# save_fig(plt.gcf(), "plots/srcs/hese_events_per_sample_skymap.pdf",
#          bbox_inches="tight", bbox_extra_artists=[leg], pad_inches=0.2)

plt.show()

## Show zoomed version before / after smoothing

In [ ]:
%%time
# Load local, unsmoothed maps
files = sorted(glob("/Users/tmenne/Downloads/hese_transient_stacking_data/" +
                    "hese_scan_maps_local/*.json.gz"))
with gzip.open(files[0]) as fp:
    src_dict = json.load(fp)
    local_map = np.array(src_dict["map"])

In [ ]:
# Render the map around the minimum
res = (400, 300)

bf_azi, bf_zen = src_dict["bf_loc"]["azi"], src_dict["bf_loc"]["zen"]
off = np.deg2rad(5)
bounds = [[bf_azi - off, bf_azi + off], [bf_zen - off, bf_zen + off]]
local, cx, cy = mplt.render_healpymap(local_map, npix=res, bounds=bounds)
# Make -log10(DeltalogLLH), so levels show sigma contours
# when approx. gaussian like
delta_local = np.amax(local) - local

# Smooth it
pdf_map = pdf_map = np.exp(local_map - np.amax(local_map))
# Smooth with a gaussian kernel
pdf_map = hp.smoothing(map_in=pdf_map, sigma=np.deg2rad(1.), verbose=False)
# Healpy smoothing may produce numerical erros, so fix them after smoothing
pdf_map[pdf_map < 0.] = 0.
smoothed = np.log(pdf_map)
# Render a new version
smoothed, cx, cy = mplt.render_healpymap(smoothed, npix=res, bounds=bounds)
delta_smoothed = np.amax(smoothed) - smoothed

In [ ]:
try:
    fs = latex_setup.make_figsize(scale=0.9, ratio=2.5)
except:
    fs = (5.2, 2.)
# fig, (axl ,axr) = plt.subplots(1, 2, figsize=fs, sharey=True)
fig = plt.figure(figsize=fs)
axl =  fig.add_axes([0.1, 0.1, 0.4, 0.9])
axr =  fig.add_axes([0.55, 0.1, 0.4, 0.9])
axcbar =  fig.add_axes([0.975, 0.1, 0.025, 0.9])

# Local coords azi, zen as stored in map
mx, my = map(lambda b: np.rad2deg(0.5 * (b[:-1] + b[1:])), [cx, cy])
xx, yy = np.meshgrid(mx, my)
levels = - scs.chi2.logsf(np.arange(0., 3.5, 0.5)**2, df=2)

# Smooth the larger contour a bit
delta_smoothed = gaussian_filter(delta_smoothed, sigma=4.)

axl.contourf(xx, yy, delta_local, levels, cmap="Greys_r", extend="max")
img = axr.contourf(xx, yy, delta_smoothed, levels, cmap="Greys_r", extend="max")
cbar = fig.colorbar(img, cax=axcbar)
cbar.set_ticks(levels[::2])
cbar.set_label(r"$-\log_{10}(\mathrm{PDF})$")

xticks = np.array([334, 336, 338, 340, 342])
yticks = np.array([54, 56, 58, 60, 62, 64])
for ax in (axl, axr):
    ax.set_xticks(xticks)
    ax.set_yticks(yticks)
    ax.set_xlim(-1 + xticks[0], xticks[-1] + 1)
    ax.set_ylim(-1 + yticks[0], yticks[-1] + 1)
    ax.set_xlabel(r"Azimuth $\phi$ in $^\circ$")
    ax.grid(ls=":")

axr.set_yticks(axl.get_yticks())
axr.set_yticklabels([])
axl.set_ylabel(r"Zenith $\theta$ in $^\circ$")

fig.tight_layout()
save_fig(fig, "plots/datasets/unsmoothed_vs_smoothed.pdf", bbox_inches="tight")
plt.show()

## Show whole map raw smoothed and truncated

In [ ]:
%%time
# Load untruncaed maps
files = sorted(glob("/Users/tmenne/Downloads/hese_transient_stacking_data/" +
                    "rawout_original_hese/hese_scan_maps/*.json.gz"))
# Just pick a single map here
with gzip.open(files[0]) as fp:
    src_dict = json.load(fp)
    untruncated = np.array(src_dict["map"])

In [ ]:
res = (400, 300)

untrunc, cx, cy = mplt.render_healpymap(untruncated, npix=res)
# Make -log10(PDF), so levels show sigma contours when approx. gaussian like
m = (untrunc > 0.)
untrunc = np.log10(untrunc)
untrunc = np.amax(untrunc) - untrunc

# Truncate map at six sigma
_, _, idx = tdu.get_pixel_in_sigma_region(pdf_map=untruncated, sigma=6.)
truncated = np.zeros_like(untruncated)
truncated[idx] = untruncated[idx]
# And make a rendered -log10(PDF) version.
trunc, cx, cy = mplt.render_healpymap(truncated, npix=res)
m = (trunc > 0.)
trunc = np.log10(trunc)
trunc = np.amax(trunc) - trunc

In [ ]:
# Untruncated on the left truncated on the right
try:
    fs = latex_setup.make_figsize(scale=0.9, ratio=2.5)
except:
    fs = (5.2, 2.)
# fig, (axl ,axr) = plt.subplots(1, 2, figsize=fs, sharey=True)
fig = plt.figure(figsize=fs)
axl =  fig.add_axes([0.1, 0.1, 0.4, 0.9])
axr =  fig.add_axes([0.55, 0.1, 0.4, 0.9])
axcbar =  fig.add_axes([0.975, 0.1, 0.025, 0.9])

mx, my = map(lambda b: np.rad2deg(0.5 * (b[:-1] + b[1:])), [cx, cy])
# Convert to equ. with ra from right to left in hours
mx = 24. - mx / 360. * 24
my = 90. - my
xx, yy = np.meshgrid(mx, my)
levels = - scs.chi2.logsf(np.arange(0., 7, 1)**2, df=2)

axl.contourf(xx, yy, untrunc, levels, cmap="Greys_r", extend="max")
img = axr.contourf(xx, yy, trunc, levels, cmap="Greys_r", extend="max")
cbar = fig.colorbar(img, cax=axcbar)
cbar.set_ticks(levels[::2])
cbar.set_label(r"$-\log_{10}(\mathrm{PDF})$")

xticks = np.linspace(0, 24, 7)    # Show as hours from right to left 
yticks = np.linspace(-90, 90, 7)  # Show as equ. coords. in [-90, +90]
xlabels = [r"${:.0f}$".format(xi) for xi in xticks][::-1]
ylabels = [r"${:.0f}$".format(yi) for yi in yticks]
for ax in (axl, axr):
    ax.set_xticks(xticks)
    ax.set_xticklabels(xlabels)
    ax.set_xlabel(r"Right ascension $\alpha$ in hours")
    ax.set_yticks(yticks)

axl.set_yticklabels(ylabels)
axr.set_yticklabels([])
axl.set_ylabel(r"Declination $\delta$ in $^\circ$")
axr.grid(ls=":")

plt.tight_layout()
save_fig(fig, "plots/datasets/untruncated_vs_truncated.pdf", bbox_inches="tight")
plt.show()

## Eff. areas and sindec distribution for true dec

In [ ]:
mc = loader.mc_loader("all")

In [ ]:
# sindec setup
bins = np.linspace(-1, 1, 100)
mids = 0.5 * (bins[:-1] + bins[1:])
Hz2mhZ = 1e3
gammas = [2.5, 2.19, 2.]
fmts = [r"$E^{{-{:.1f}}}$", r"$E^{{-{:.2f}}}$", r"$E^{{-{:.0f}}}$"]
labels = [f.format(g) for f, g in zip(fmts, gammas)]
cs = ["0.6", "0.3", "0."]
phi0 = 1e-18
E0 = 1e5

# eff A setup
cm2tom2 = 1e4
gevtotev = 1e3
e_bins = np.logspace(2, 9, 100)  # in GeV
x = np.logspace(e_bins[0], e_bins[-1], 500)
e_mids = 0.5 * (e_bins[:-1] + e_bins[1:])
decs_lo = np.deg2rad([-90, 30, -5, -90])
decs_hi = np.deg2rad([ 90, 90, 30,  -5])

try:
    labsize = latex_setup.latex_settings["axes.labelsize"] - 3
    fs = latex_setup.make_figsize(scale=0.95, ratio=0.8)
except:
    fs = (5, 2.5)

# Make eff. area on the left and sindec on the right
nrows, ncols = 4, 2
fig, axs = plt.subplots(nrows, ncols, figsize=fs)
for row, name in enumerate(sorted(mc.keys())):
    axl, axr = axs[row, 0], axs[row, 1]
    
    mc_i = mc[name]
    Enu = mc_i["trueE"]    # in GeV
    dec = mc_i["trueDec"]  # in rad
    ow = mc_i["ow"]

    # sindec plot for multiple flux weightings
    for i, gamma in enumerate(gammas):  # phi0 * (E/E0)**-gamma
        # Rate in mHz
        w = Hz2mhZ * phi0 * ow * (mc_i["trueE"] / E0)**-gamma
        h, b = np.histogram(np.sin(mc_i["trueDec"]), bins=bins, weights=w,
                            density=False)
        err, _ = np.histogram(np.sin(mc_i["trueDec"]), bins=bins, weights=w**2,
                              density=False)
        axr.plot(b, np.r_[h[0], h], drawstyle="steps-pre", c=cs[i], lw=2,
                 label=labels[i])
        axr.errorbar(
            mids, h, yerr=err, fmt=",", color=cs[i], zorder=-10, lw=1)

    if row == nrows - 1:
        axr.set_xlabel(r"$\sin(\delta_\nu)$", fontsize=labsize)
    axr.set_ylabel("Rate per bin in mHz", fontsize=labsize - 1)
    axr.set_yscale("log")
    axr.set_xlim(-1, 1)
    axr.set_ylim(5e-7, 3e-4)
    axr.legend(loc="lower right", prop={"size": labsize - 4})
    axr.grid(ls=":")
    
    ###########################################################################
    # Make eff. area in three decl. regions and full sky
    ls = ["-", "-", "-", "-"]
    cs_effA = ["#353132", "0.3", "0.5", "0.75"]
    _s = r"$\SI{{{:.0f}}}{{\degree}} \leq \delta \leq \SI{{{:.0f}}}{{\degree}}$"
    labels_effA = [_s.format(*np.rad2deg(dec_rng))
                   for dec_rng in zip(decs_lo, decs_hi)]
    labels_effA[0] = "All-sky"

    for i, (dec_lo, dec_hi) in enumerate(zip(decs_lo, decs_hi)):
        mask = np.logical_and(dec >= dec_lo, dec < dec_hi)
        Enu_i, w_i = Enu[mask], ow[mask]
        h, _ = np.histogram(Enu_i, bins=e_bins, weights=w_i, normed=False)
        solid_angle = 2 * np.pi * (np.sin(dec_hi) - np.sin(dec_lo))
        norm = np.diff(e_bins) * solid_angle * cm2tom2
        err, _ = np.histogram(Enu_i, bins=e_bins, weights=w_i**2, normed=False)
        h = h / norm
        err = np.sqrt(err) / norm
        
        # truncate manually for pgf
        ymin = 1e-3
        h[h < ymin] = ymin / 10.

        axl.plot(e_bins / gevtotev, np.r_[h[0], h], drawstyle="steps-pre",
                 c=cs_effA[i], label=labels_effA[i], ls=ls[i])
        axl.errorbar(e_mids / gevtotev, h, yerr=err, fmt=",",
                     color=cs_effA[i], zorder=-10, lw=1)
        
    axl.set_xscale("log")
    axl.set_yscale("log")
    axl.set_xlim(e_bins[0] / gevtotev, e_bins[-1] / gevtotev)
    axl.set_ylim(ymin, 1e4)
    axl.grid(ls=":")

    if row == nrows - 1:
        axl.set_xlabel(r"$E_\nu$ in $\si{\TeV}$", fontsize=labsize)
    axl.set_ylabel(r"$A_\mathrm{eff}^{\nu_\mu+\bar{\nu}_\mu}$ in " +
                   r"$\si{\m\squared}$", fontsize=labsize)
    axl.legend(loc="lower left", prop={"size": labsize - 4},
               bbox_to_anchor=(0.4, 0.01))
    axl.set_title("Sample {}".format(key2label(name)), fontsize=labsize + 1)

#     fig.suptitle("Sample {}".format(key2label(name)), y=1)
fig.tight_layout()
save_fig(plt.gcf(), "plots/datasets/effA_and_sindec.pdf".format(name),
         bbox_inches="tight")

plt.show()

Old side-by.side standalone plots

In [ ]:
# sindec setup
bins = np.linspace(-1, 1, 100)
mids = 0.5 * (bins[:-1] + bins[1:])
Hz2mhZ = 1e3
gammas = [2.5, 2.19, 2.]
fmts = [r"$E^{{-{:.1f}}}$", r"$E^{{-{:.2f}}}$", r"$E^{{-{:.0f}}}$"]
labels = [f.format(g) for f, g in zip(fmts, gammas)]
cs = ["0.6", "0.3", "0."]
phi0 = 1e-18
E0 = 1e5

# eff A setup
cm2tom2 = 1e4
gevtotev = 1e3
e_bins = np.logspace(2, 9, 100)  # in GeV
x = np.logspace(e_bins[0], e_bins[-1], 500)
e_mids = 0.5 * (e_bins[:-1] + e_bins[1:])
decs_lo = np.deg2rad([-90, 30, -5, -90])
decs_hi = np.deg2rad([ 90, 90, 30,  -5])

try:
    fs = latex_setup.make_figsize(ratio=2.5)
except:
    fs = (5, 2.5)

# Make eff. area on the left and sindec on the right
for name in mc.keys()[:]:
    fig, (axl, axr) = plt.subplots(1, 2, figsize=fs)
    mc_i = mc[name]
    Enu = mc_i["trueE"]    # in GeV
    dec = mc_i["trueDec"]  # in rad
    ow = mc_i["ow"]

    # sindec plot for multiple flux weightings
    for i, gamma in enumerate(gammas):  # phi0 * (E/E0)**-gamma
        # Rate in mHz
        w = Hz2mhZ * phi0 * ow * (mc_i["trueE"] / E0)**-gamma
        h, b = np.histogram(np.sin(mc_i["trueDec"]), bins=bins, weights=w,
                            density=False)
        err, _ = np.histogram(np.sin(mc_i["trueDec"]), bins=bins, weights=w**2,
                              density=False)
        axr.plot(b, np.r_[h[0], h], drawstyle="steps-pre", c=cs[i], lw=2,
                 label=labels[i])
        axr.errorbar(
            mids, h, yerr=err, fmt=",", color=cs[i], zorder=-10, lw=1)

    axr.set_xlabel(r"$\sin(\delta_\nu)$",
                   fontsize=latex_setup.latex_settings["axes.labelsize"] - 2)
    axr.set_ylabel("Rate per bin in mHz",
                   fontsize=latex_setup.latex_settings["axes.labelsize"] - 2)
    axr.set_yscale("log")
    axr.set_xlim(-1, 1)
    axr.set_ylim(5e-7, 3e-4)
    axr.legend(loc="lower right",
               prop={"size": latex_setup.latex_settings["legend.fontsize"] - 3})
    
    ###########################################################################

    # Make eff. area in three decl. regions and full sky
    ls = ["-", "-", "-", "-"]
    cs_effA = ["#353132", "0.3", "0.5", "0.75"]
    _s = r"$\SI{{{:.0f}}}{{\degree}} \leq \delta \leq \SI{{{:.0f}}}{{\degree}}$"
    labels_effA = [_s.format(*np.rad2deg(dec_rng))
                   for dec_rng in zip(decs_lo, decs_hi)]
    labels_effA[0] = "All-sky"

    for i, (dec_lo, dec_hi) in enumerate(zip(decs_lo, decs_hi)):
        mask = np.logical_and(dec >= dec_lo, dec < dec_hi)
        Enu_i, w_i = Enu[mask], ow[mask]
        h, _ = np.histogram(Enu_i, bins=e_bins, weights=w_i, normed=False)
        solid_angle = 2 * np.pi * (np.sin(dec_hi) - np.sin(dec_lo))
        norm = np.diff(e_bins) * solid_angle * cm2tom2
        err, _ = np.histogram(Enu_i, bins=e_bins, weights=w_i**2, normed=False)
        h = h / norm
        err = np.sqrt(err) / norm
        
        # truncate manually for pgf
        ymin = 1e-3
        h[h < ymin] = ymin / 10.

        axl.plot(e_bins / gevtotev, np.r_[h[0], h], drawstyle="steps-pre",
                 c=cs_effA[i], label=labels_effA[i], ls=ls[i])
        axl.errorbar(e_mids / gevtotev, h, yerr=err, fmt=",",
                     color=cs_effA[i], zorder=-10, lw=1)
        
    axl.set_xscale("log")
    axl.set_yscale("log")
    axl.set_xlim(e_bins[0] / gevtotev, e_bins[-1] / gevtotev)
    axl.set_ylim(ymin, 1e4)
    axl.grid()

    axl.set_xlabel(r"$E_\nu$ in $\si{\TeV}$",
                   fontsize=latex_setup.latex_settings["axes.labelsize"] - 2)
    axl.set_ylabel(r"$A_\mathrm{eff}^{\nu_\mu+\bar{\nu}_\mu}$ in " +
                   r"$\si{\m\squared}$",
                   fontsize=latex_setup.latex_settings["axes.labelsize"] - 2)
    axl.legend(prop={"size": latex_setup.latex_settings["legend.fontsize"] - 3})

    fig.suptitle("Sample {}".format(key2label(name)), y=1)
    fig.tight_layout()
    save_fig(plt.gcf(), "plots/datasets/effA_and_sindec_{}.pdf".format(name),
             bbox_inches="tight")
    
    plt.show()

Previous standalone plots

In [ ]:
bins = np.linspace(-1, 1, 100)
mids = 0.5 * (bins[:-1] + bins[1:])
Hz2mhZ = 1e3
gammas = [2.5, 2.19, 2.]
fmts = [r"$E^{{-{:.1f}}}$", r"$E^{{-{:.2f}}}$", r"$E^{{-{:.0f}}}$"]
labels = [f.format(g) for f, g in zip(fmts, gammas)]
cs = ["0.6", "0.3", "0."]
phi0 = 1e-18
E0 = 1e5

# Make eff. area on the left and sindec on the right
for name in mc.keys():
    mc_i = mc[name]
    for i, gamma in enumerate(gammas):  # phi0 * (E/E0)**-gamma
        # Rate in mHz
        w = Hz2mhZ * phi0 * mc_i["ow"] * (mc_i["trueE"] / E0)**-gamma
        h, b = np.histogram(np.sin(mc_i["trueDec"]), bins=bins, weights=w,
                            density=False)
        err, _ = np.histogram(np.sin(mc_i["trueDec"]), bins=bins, weights=w**2,
                              density=False)
        plt.plot(b, np.r_[h[0], h], drawstyle="steps-pre", c=cs[i], lw=2,
                 label=labels[i])
        plt.errorbar(
            mids, h, yerr=err, fmt=",", color=cs[i], zorder=-10, lw=1)

    plt.xlabel(r"$\sin(\delta_\nu)$")
    plt.ylabel("Rate per bin in mHz")
    plt.title("Sample {}".format(key2label(name)))

    plt.yscale("log")
    plt.xlim(-1, 1)
    plt.ylim(5e-7, 3e-4)
    plt.legend(loc="upper left")
    
    save_fig(plt.gcf(), "plots/datasets/true_sindec_{}.pdf".format(name),
             bbox_inches="tight")
    
    plt.show()

In [ ]:
cm2tom2 = 1e4
gevtotev = 1e3

e_bins = np.logspace(2, 9, 100)  # in GeV
x = np.logspace(e_bins[0], e_bins[-1], 500)
e_mids = 0.5 * (e_bins[:-1] + e_bins[1:])
decs_lo = np.deg2rad([-90, 30, -5, -90])
decs_hi = np.deg2rad([ 90, 90, 30,  -5])

for name, mc_i in mc.items():
    mc_i = mc[name]
    Enu = mc_i["trueE"]    # in GeV
    dec = mc_i["trueDec"]  # in rad
    w = mc_i["ow"]

    # Make eff. area in three decl. regions and full sky
    ls = ["-", "-", "-", "-"]
    cs = ["#353132", "0.3", "0.5", "0.75"]
    _s = r"$\SI{{{:.0f}}}{{\degree}} \leq \delta \leq \SI{{{:.0f}}}{{\degree}}$"
    labels = [_s.format(*np.rad2deg(dec_rng))
              for dec_rng in zip(decs_lo, decs_hi)]
    labels[0] = "All-sky"

    for i, (dec_lo, dec_hi) in enumerate(zip(decs_lo, decs_hi)):
        mask = np.logical_and(dec >= dec_lo, dec < dec_hi)
        Enu_i, w_i = Enu[mask], w[mask]
        h, _ = np.histogram(Enu_i, bins=e_bins, weights=w_i, normed=False)
        solid_angle = 2 * np.pi * (np.sin(dec_hi) - np.sin(dec_lo))
        norm = np.diff(e_bins) * solid_angle * cm2tom2
        err, _ = np.histogram(Enu_i, bins=e_bins, weights=w_i**2, normed=False)
        h = h / norm
        err = np.sqrt(err) / norm
        
        # truncate manually for pgf
        ymin = 1e-3
        h[h < ymin] = ymin / 10.

        plt.plot(e_bins / gevtotev, np.r_[h[0], h], drawstyle="steps-pre",
                 c=cs[i], label=labels[i], ls=ls[i])
        plt.errorbar(e_mids / gevtotev, h, yerr=err, fmt=",",
                     color=cs[i], zorder=-10, lw=1)
        
    plt.xscale("log")
    plt.yscale("log")
    plt.xlim(e_bins[0] / gevtotev, e_bins[-1] / gevtotev)
    plt.ylim(ymin, 1e4)
    plt.grid()

    plt.xlabel(r"True neutrino energy $E_\nu$ in $\si{\TeV}$")
    plt.ylabel(r"Eff. area $A_\mathrm{eff}^{\nu_\mu+\bar{\nu}_\mu}$ in " +
               r"$\si{\m\squared}$")
    plt.title("Sample {}".format(key2label(name)))
    plt.legend()

    save_fig(plt.gcf(), "plots/datasets/eff_area_{}.pdf".format(name),
             bbox_inches="tight")
    
    plt.show()

# Chapter: Time Independent Analysis

Load saved ana objects from the analysis_plots.ipynb for fast loading.

In [ ]:
with open("/Users/tmenne/Downloads/multi_bg_inj.pickle", "r") as fp:
    multi_bg_inj = dill.load(fp)
    
with open("/Users/tmenne/Downloads/multi_sig_inj.pickle", "r") as fp:
    multi_sig_inj = dill.load(fp)
    
with open("/Users/tmenne/Downloads/multi_llh.pickle", "r") as fp:
    multi_llh = dill.load(fp)
    
with open("/Users/tmenne/Downloads/ana.pickle", "r") as fp:
    ana = dill.load(fp)

## Rate plots

In [ ]:
dt0, dt1 = loader.time_window_loader(-1)
src_dicts = loader.source_list_loader("all")
exp_off_dict = loader.off_data_loader("all")
runlist_dict = loader.runlist_loader("all")
rate_recs_dict = {}
srcs_recs_dict = {}
for key, rl in runlist_dict.items():
    rate_recs_dict[key] = tdu.make_rate_records(
        ev_runids=exp_off_dict[key]["Run"], run_list=rl)
    srcs_recs_dict[key] = tdu.make_src_records(src_dicts[key], dt0=dt0, dt1=dt1)

In [ ]:
# Plot rates and sources for each sample
_Hz2mHz = 1e3
max_y = _Hz2mHz * 0.01
try:
    fs = latex_setup.make_figsize(ratio=2.)
except:
    fs = (5, 2.5)
fig, ax = plt.subplots(1, 1, figsize=fs)
    
for i, key in enumerate(sorted(exp_off_dict.keys())):
    # Get the sine model
    bginj = multi_bg_inj.injs[key]
    rf = bginj._spl_info["allsky_rate_func"]
    pars = bginj._spl_info["allsky_best_params"]
    
    mids = 0.5 * (rate_recs_dict[key]["start_mjd"] +
                  rate_recs_dict[key]["stop_mjd"])

    min_x = np.amin(rate_recs_dict[key]["start_mjd"])
    max_x = np.amax(rate_recs_dict[key]["stop_mjd"])
    
    t = np.linspace(min_x, max_x, 500)
    rf_vals = rf.fun(t, pars) * _Hz2mHz
      
    ax.errorbar(mids, _Hz2mHz * rate_recs_dict[key]["rate"],
                yerr=_Hz2mHz * rate_recs_dict[key]["rate_std"],
                fmt=",", color="C7", alpha=0.15, zorder=-10)   
    ax.plot(t, rf_vals, c="#353132")
        
    ax.vlines(srcs_recs_dict[key]["time"], 0, 1, colors="#353132",
               linestyles="-", label=key, lw=1)
    if i > 0:
        ax.axvline(np.amin(rate_recs_dict[key]["start_mjd"]),
                   c="#353132", ls="-.")
    
    ax.text(0.5 * (min_x + max_x), max_y + 0.05,
            key2label(key) + r"\phantom{,}",
            fontsize=latex_setup.latex_settings["axes.labelsize"] - 2,
            ha="center", va="bottom", color="#353132")
    
ax.set_xlim(np.amin(rate_recs_dict["IC79"]["start_mjd"]),
            np.amax(rate_recs_dict["IC86_2015"]["stop_mjd"]))
ax.set_ylim(0, max_y)
ax.set_xlabel("Time in MJD days")
ax.set_ylabel("Rate in mHz")

save_fig(plt.gcf(), "plots/time_dep/rates_all.pdf", bbox_inches="tight")
plt.show()

## Rate plots per bin

In [ ]:
dt0, dt1 = loader.time_window_loader(-1)
src_dicts = loader.source_list_loader("all")
exp_off_dict = loader.off_data_loader("all")
runlist_dict = loader.runlist_loader("all")
rate_recs_dict = {}
srcs_recs_dict = {}
for key, rl in runlist_dict.items():
    rate_recs_dict[key] = tdu.make_rate_records(
        ev_runids=exp_off_dict[key]["Run"], run_list=rl)
    srcs_recs_dict[key] = tdu.make_src_records(src_dicts[key], dt0=dt0, dt1=dt1)

In [ ]:
MAX_RECS = 200  # Constrain no. of points drawn to be able to plot it wiht tex
alpha = 0.75
Hz2mHz = 1e3
try:
    # Almost full page ratio
    fs = latex_setup.make_figsize(scale=0.95, ratio=1.)
    legfntsize = latex_setup.latex_settings["legend.fontsize"] - 3
    labfntsize = latex_setup.latex_settings["axes.labelsize"] - 3
except:
    fs = (5, 7.15)

for key, bg_inj in sorted(multi_bg_inj._injs.items())[:]:
    print("Making plots for sample: '{}'".format(key))
    runids = exp_off_dict[key]["Run"]
    sindec = np.sin(exp_off_dict[key]["dec"])
    bg_inj_opts = bg_inj.inj_opts
    sd_bins = bg_inj_opts["sindec_bins"]
    rate_fun = bg_inj._spl_info["allsky_rate_func"]

    # Make a plot grid
    nplots, nrows, ncols = 20, 5, 4
    assert ncols * nrows == nplots
    assert nplots == len(sd_bins) - 1
    fig, ax = plt.subplots(nrows=nrows, ncols=ncols, figsize=fs,
                           sharex=True, sharey=True)
    
    for i, (lo, hi) in enumerate(zip(sd_bins[:-1], sd_bins[1:])):
        mask = (sindec >= lo) & (sindec < hi)
        recs = tdu.make_rate_records(run_list=runlist_dict[key],
                                     ev_runids=runids[mask])
        rebin = tdu.rebin_rate_rec(
            rate_rec=recs, bins=bg_inj_opts["rate_rebins"],
            ignore_zero_runs=True)
        rates, bins, rate_std, _ = rebin
        
        mids = 0.5 * (recs["start_mjd"] + recs["stop_mjd"])
        xmin, xmax = np.amin(recs["start_mjd"]), np.amax(recs["stop_mjd"])
        diff = recs["stop_mjd"] - recs["start_mjd"]
        t = np.linspace(bins[0], bins[-1], 200)
        
        amp, base = bg_inj._spl_info["best_pars"][i]
        pars = (amp, bg_inj._spl_info["allsky_best_params"][1], base)
        lab = r"${:.2f} \leq \sin(\delta) < {:.2f}$".format(lo, hi)
        
        # Plot it
        CONSTR = len(recs) // MAX_RECS
        sli = slice(0, -1, CONSTR)
        print("Only plotting every {} element".format(CONSTR))
        row, col = idx2rowcol(i, ncols=ncols)
        ax_ = ax[row, col]
        
        ax_.errorbar(mids[sli], recs["rate"][sli] * Hz2mHz,
                     xerr=diff[sli], yerr=recs["rate_std"][sli] * Hz2mHz,
                     fmt=",", color="0.6", alpha=alpha, label=lab, zorder=1)
        ax_.plot(
            bins, np.r_[rates[0], rates] * Hz2mHz,
            drawstyle="steps-pre", color="w", lw=2)
        ax_.plot(t, rate_fun.fun(t, pars) * Hz2mHz, color="#353132", lw=1)

        ax_.set_ylim(0, 1.)
        ax_.set_xlim(xmin, xmax)
        if col == 0:
            ax_.set_ylabel("Rate in mHz", fontsize=labfntsize)
        if row == nrows - 1:
            ax_.set_xlabel("MJD", fontsize=labfntsize)
        ax_.text(x=.5, y=1.025, s=lab, transform=ax_.transAxes,
                 va="bottom", ha="center", fontsize=legfntsize)
#         ax_.text(x=1.025, y=0.5, s=lab, transform=ax_.transAxes,
#                  rotation=90, va="center", ha="left",
#                  fontsize=legfntsize)
    fig.tight_layout()
    save_fig(fig, "plots/time_dep/rates_per_bin_{}.pdf".format(key),
             bbox_inhces="tight")
    plt.show()

## BG PDF sine param splines

In [ ]:
x = np.linspace(-1, 1, 100)
Hz2mHz = 1e3
lbfntsize = latex_setup.latex_settings["axes.labelsize"] - 4

try:
    fs = latex_setup.make_figsize(scale=0.95, ratio=0.8)  # Ful page
except:
    fs = (5, 2.)
    
ylabels = {
    "amp": r"Amplitude in \si{\milli\Hz}",
    "base": r"Baseline in \si{\milli\Hz}",
}

nrows, ncols = 4, 2
fig, axs = plt.subplots(nrows, ncols, figsize=fs)

for row, (key, bg_inj) in enumerate(sorted(multi_bg_inj.injs.items())):
#     fig, (axl, axr) = plt.subplots(1, 2, figsize=fs)
    axl, axr = axs[row, 0], axs[row, 1]
    
    print("Plotting for sample '{}'".format(key))
    bg_inj_opts = bg_inj.inj_opts
    bins = bg_inj_opts["sindec_bins"]
    mids = 0.5 * (bins[:-1] + bins[1:])
    print("Allsky best params: " + arr2str(
        bg_inj._spl_info["allsky_best_params"]))
   
    for n, ax in zip(["amp", "base"], (axl, axr)):
        spl = bg_inj._spl_info["param_splines"][n]
        vals = np.copy(bg_inj._spl_info["best_pars_norm"][n])
        err = np.copy(bg_inj._spl_info["best_stddevs_norm"][n])

        ax.plot(x, spl(x) * Hz2mHz, color="#353132", label=key2label(key))
        ax.errorbar(mids, vals * Hz2mHz, yerr=err * Hz2mHz,
                    xerr=np.diff(bins) / 2., lw=1.,
                    fmt=",", color="0.5", zorder=10)

        if n == "amp":
            ax.axhline(0, 0, 1, color="#353132", ls="-", lw=1)
            ax.set_ylim(-0.5, 0.1)
        else:
            ax.set_ylim(0, 6)
            
        ax.grid(ls=":")
        ax.set_axisbelow(True)
        if row == nrows - 1:
            ax.set_xlabel(r"$\sin(\delta)$", fontsize=lbfntsize)
        ax.set_ylabel(ylabels[n], fontsize=lbfntsize)
        ylim = ax.set_xlim(-1, 1)
    if row == nrows - 1:
        axr.legend(loc="lower center""", prop={"size": lbfntsize})
    else:
        axr.legend(loc="upper center""", prop={"size": lbfntsize})

fig.tight_layout()
save_fig(fig, "plots/time_dep/sine_param_splines.pdf",
         bbox_inches="tight")
plt.show()

Old, single standalone plots

In [ ]:
x = np.linspace(-1, 1, 100)
Hz2mHz = 1e3
lbfntsize = latex_setup.latex_settings["axes.labelsize"] - 2

try:
    fs = latex_setup.make_figsize(ratio=2.5)
except:
    fs = (5, 2.)
    
ylabels = {
    "amp": r"Amplitude in \si{\milli\Hz}",
    "base": r"Baseline in \si{\milli\Hz}",
}

for key, bg_inj in sorted(multi_bg_inj.injs.items())[:]:
    fig, (axl, axr) = plt.subplots(1, 2, figsize=fs)
    
    print("Plotting for sample '{}'".format(key))
    bg_inj_opts = bg_inj.inj_opts
    bins = bg_inj_opts["sindec_bins"]
    mids = 0.5 * (bins[:-1] + bins[1:])
    print("Allsky best params: " + arr2str(
        bg_inj._spl_info["allsky_best_params"]))
   
    for n, ax in zip(["amp", "base"], (axl, axr)):
        spl = bg_inj._spl_info["param_splines"][n]
        vals = np.copy(bg_inj._spl_info["best_pars_norm"][n])
        err = np.copy(bg_inj._spl_info["best_stddevs_norm"][n])

        ax.plot(x, spl(x) * Hz2mHz, color="#353132")
        ax.errorbar(mids, vals * Hz2mHz, yerr=err * Hz2mHz,
                    xerr=np.diff(bins) / 2., lw=1.,
                    fmt=",", color="0.3", zorder=1)

        if n == "amp":
            ax.axhline(0, 0, 1, color="#353132", ls="-")
            ax.set_ylim(-0.5, 0.1)
        else:
            ax.set_ylim(0, 6)
            
        ax.grid()
        ax.set_axisbelow(True)
        ax.set_xlabel(r"$\sin(\delta)$", fontsize=lbfntsize)
        ax.set_ylabel(ylabels[n], fontsize=lbfntsize)
        ylim = ax.set_xlim(-1, 1)

    fig.tight_layout()
    save_fig(fig, "plots/time_dep/sine_param_splines_{}.pdf".format(key),
             bbox_inches="tight")
    plt.show()

## 2D energy PDF per sample

In [ ]:
try:
    lbfntsize = latex_setup.latex_settings["axes.labelsize"] - 3
    fs = latex_setup.make_figsize(scale=0.9)
except:
    fs = (5, 2.)
    
nrows, ncols = 2, 2
fig, axs = plt.subplots(nrows, ncols, figsize=fs, sharex=True, sharey=True)

for i, (key, grb_mod_i) in enumerate(multi_llh.model.items()):
#     fig, ax = plt.subplots(1, 1, figsize=fs)
    row, col = idx2rowcol(idx=i, ncols=ncols)
    ax = axs[row, col]
    # Scan the PDF
    xbins = grb_mod_i.energy_opts["bins"][0]
    ybins = grb_mod_i.energy_opts["bins"][1]
    
    xlo, xhi = np.amin(xbins), np.amax(xbins)
    ylo, yhi = np.amin(ybins), np.amax(ybins)

    x = np.linspace(xlo, xhi, 400)
    y = np.linspace(ylo, yhi, 400)
    xx, yy = np.meshgrid(x, y)

    xmids, ymids = map(lambda b: 0.5 * (b[:-1] + b[1:]), [x, y])
    XX, YY = map(np.ravel, np.meshgrid(xmids, ymids))
    pts = np.vstack((XX, YY)).T

    zz = grb_mod_i._soverb_energy(XX, YY)
    zz = zz.reshape(len(xmids), len(ymids))

    # Contour level plot
    levels = np.linspace(-3, 3, 7)
    img = ax.contourf(
        xmids, ymids, np.log10(zz), levels, extend="both", cmap="Greys")
    cb = fig.colorbar(img, ax=ax)
    if col == ncols - 1:
        cb.set_label(r"$\log_{10}\left( S^E / B^E \right)$", fontsize=lbfntsize)

    ax.set_xlim(-1, 1)
    ax.set_ylim(2, 9)
    ax.set_title(key2label(key), fontsize=lbfntsize + 1)
    if row == nrows - 1:
        ax.set_xlabel(r"$\sin(\delta)$", fontsize=lbfntsize)
    if col == 0:
        ax.set_ylabel(r"$\log_{10}(E_\mathrm{proxy})$", fontsize=lbfntsize)

fig.tight_layout(w_pad=0.25, h_pad=0.15)
save_fig(fig, "plots/time_dep/energy_pdfs.pdf", bbox_inches="tight")
plt.show()

Old standalone plots

In [ ]:
try:
    fs = latex_setup.make_figsize()
except:
    fs = (5., 2.)

for key, grb_mod_i in multi_llh.model.items()[:]:
    fig, ax = plt.subplots(1, 1, figsize=fs)
    # Scan the PDF
    xbins = grb_mod_i.energy_opts["bins"][0]
    ybins = grb_mod_i.energy_opts["bins"][1]
    
    xlo, xhi = np.amin(xbins), np.amax(xbins)
    ylo, yhi = np.amin(ybins), np.amax(ybins)

    x = np.linspace(xlo, xhi, 400)
    y = np.linspace(ylo, yhi, 400)
    xx, yy = np.meshgrid(x, y)

    xmids, ymids = map(lambda b: 0.5 * (b[:-1] + b[1:]), [x, y])
    XX, YY = map(np.ravel, np.meshgrid(xmids, ymids))
    pts = np.vstack((XX, YY)).T

    zz = grb_mod_i._soverb_energy(XX, YY)
    zz = zz.reshape(len(xmids), len(ymids))

    # Contour level plot
    levels = np.linspace(-3, 3, 7)
    img = ax.contourf(
        xmids, ymids, np.log10(zz), levels, extend="both", cmap="Greys")
    cb = fig.colorbar(img, ax=ax)
    cb.set_label(r"$\log_{10}(S^E / B^E)$")

    ax.set_xlim(-1, 1)
    ax.set_ylim(2, 9)
    ax.set_xlabel(r"$\sin(\delta)$")
    ax.set_ylabel(r"$\log_{10}(E_\mathrm{proxy})$")

    fig.tight_layout()
    save_fig(fig, "plots/time_dep/energy_pdf_{}.pdf".format(key),
             bbox_inches="tight")
    plt.show()

## BG PDF per sample

Simply plot all PDFs per sample above each other to give an idea of the fluctuations

In [ ]:
x = np.linspace(-1, 1, 200)
try:
    fs = latex_setup.make_figsize(ratio=1.25)
except:
    fs = (5., 2.)

fig, axs = plt.subplots(2, 2, figsize=fs, sharex=True, sharey=True)
key2ax = {
    "IC79": axs[0, 0],
    "IC86_2011": axs[0, 1],
    "IC86_2012-2014": axs[1, 0],
    "IC86_2015": axs[1, 1],
}

for key, bg_inj in sorted(multi_bg_inj._injs.items())[:]:
    ax = key2ax[key]
    bins = bg_inj.inj_opts["sindec_bins"]
    mids = 0.5 * (bins[:-1] + bins[1:])
    h, _ = np.histogram(np.sin(bg_inj._X["dec"]), bins=bins, density=False)
    norm = np.diff(bins) * np.sum(h)
    stddev = np.sqrt(h) / norm
    h = h / norm
    ax.plot(bins, np.r_[h[0], h], drawstyle="steps-pre", color="0.6", lw=2)
    ax.errorbar(mids, h, yerr=stddev, color="0.75", lw=2, fmt=",", zorder=1.)
    for spl in bg_inj._spl_info["sin_dec_pdf_splines"]:
        ax.plot(x, spl(x), c="#353132", lw=.5)
        
    if key in ["IC79", "IC86_2012-2014"]:
        ax.set_ylabel("PDF")
    if key in ["IC86_2012-2014", "IC86_2015"]:
        ax.set_xlabel(r"$\sin(\delta)$")
    ax.set_xlim(-1, 1)
    ax.set_ylim(0, 1)
    ax.grid(ls=":")
    ax.set_axisbelow(True)
    ax.set_title(key2label(key) + ": {} sources".format(len(bg_inj._spl_info["sin_dec_pdf_splines"])),
                 fontsize=latex_setup.latex_settings["axes.labelsize"])

fig.tight_layout()
save_fig(fig, "plots/time_dep/bg_pdf_per_sample.pdf",
         bbox_inches="tight")
plt.show()

## Stacking weight splines per sample

Simply plot all PDFs per sample above each other to give an idea of the fluctuations

In [ ]:
x = np.linspace(-1, 1, 200)
try:
    fs = latex_setup.make_figsize(ratio=1.25)
except:
    fs = (5., 2.)

fig, axs = plt.subplots(2, 2, figsize=fs, sharex=True, sharey=True)
key2ax = {
    "IC79": axs[0, 0],
    "IC86_2011": axs[0, 1],
    "IC86_2012-2014": axs[1, 0],
    "IC86_2015": axs[1, 1],
}

for key, llh_mod in sorted(multi_llh._llhs.items())[:]:
    ax = key2ax[key]
    
    spl = llh_mod._model._spl_info["mc_sin_dec_pdf_spline"]
    srcs_sin_dec = np.sin(llh_mod._model.srcs["dec"])
    stack_w = llh_mod._model.get_args()["src_w_dec"]
    norm = spl.integral(-1, 1)
#     norm = np.sum(stack_w)
    stack_w = stack_w / norm
    
    ax.plot(x, spl(x) / norm, c="#353132")
    ax.scatter(srcs_sin_dec, stack_w, marker="d", c="#353132", s=25)
        
    if key in ["IC79", "IC86_2012-2014"]:
        ax.set_ylabel("PDF")
    if key in ["IC86_2012-2014", "IC86_2015"]:
        ax.set_xlabel(r"$\sin(\delta_\nu)$")
    ax.set_xlim(-1, 1)
    ax.set_ylim(0, 1.5)
    ax.grid(ls=":")
    ax.set_axisbelow(True)
    ax.set_title(key2label(key),
                 fontsize=latex_setup.latex_settings["axes.labelsize"])

fig.tight_layout()
save_fig(fig, "plots/time_dep/stacking_src_w_per_sample.pdf",
         bbox_inches="tight")
plt.show()

## BG only TS

In [ ]:
%%time
bg_pdfs = loader.bg_pdf_loader(idx="all")
tw_ids = sorted(bg_pdfs.keys())

Scan best threshold for scan plots

In [ ]:
%%time
threshs = []
res = []

for tw_id in tw_ids:
    print("# Time window {}".format(tw_id))      
    # Scan in a range with still good statistics, but leave the really good
    # statistics part to the empirical PDF
    bg_pdfs[tw_id].fit_thresh(np.amax(bg_pdfs[tw_id].data))
    lo, hi = bg_pdfs[tw_id].ppf(q=100. * sigma2prob([3., 5.5]))
    threshs.append(np.arange(lo, hi, 0.1))
    # Best fit: KS test p-value is larger than `pval_thresh` the first time
    pval_thresh = 0.5
    res_i = tdu.scan_best_thresh(
        bg_pdfs[tw_id], threshs[-1], pval_thresh=pval_thresh)
    best_thresh, best_idx, pvals, scales = res_i
    res.append(res_i)

    best_thresh_sigma = prob2sigma(bg_pdfs[tw_id].cdf(best_thresh))
    print("Best thresh at TS = {:.2f}, {:.2f} sigma".format(
        best_thresh, best_thresh_sigma[0]))

In [ ]:
# For the other notebook...
idx = np.array([res_i[1] for res_i in res])
locs = np.array([res_i[0] for res_i in res])
scales = np.array([res_i[3][idx[i]] for i, res_i in enumerate(res)])

TS left, KS right for 4x(5x2) plots for the appendix and a single (1x2) for inline.

In [ ]:
def plot_row(axl, axr, res, threshs, bg_pdfs, tw_id, x):
    best_thresh, best_idx, pvals, scales = res[tw_id]
    lblfntsize = latex_setup.latex_settings["axes.labelsize"] - 3
    legfntsize = latex_setup.latex_settings["axes.labelsize"] - 4
    
    # ## Plot BG TS left
    c_hist, c_emp = "0.65", "#353132"
    y = bg_pdfs[tw_id].pdf(x, dx=0.25)
    h, bins, err, norm = bg_pdfs[tw_id].data_hist(density=True, dx=0.25,
                                                  which="all")  
    m = 0.5 * (bins[:-1] + bins[1:])
    # Clip manually for PGF backend
    ymin = np.amin(h[h>0] / 2.)
    h[h < ymin / 10.] = ymin / 10.
    y[y < ymin / 10.] = ymin / 10.

    axl.plot(bins, np.r_[h[0], h], drawstyle="steps-pre", color=c_hist)
    axl.errorbar(m, h, yerr=err, fmt=",", color=c_hist, zorder=-10)
    axl.axvline(best_thresh, 0, 1, c=c_emp, ls="--")
    axl.plot(x, y, color=c_emp)

    axl.set_xlabel(r"Test statistic $-2\ln\Lambda$", fontsize=lblfntsize)
    axl.set_ylabel("PDF", fontsize=lblfntsize)
    axl.set_xlim(0, 35)
    axl.set_ylim(ymin, 1)
    axl.set_yscale("log")
    axl.set_title("Time window {}".format(tw_id + 1), fontsize=lblfntsize)
#     axl.legend(loc="upper right", fontsize=legfntsize)

    # ## Plot KS test right
    # pvalue guides
    axr.axhline(0.5, 0, 1, ls=":", color="0.6", lw=1)
    axr.axhline(1, 0, 1, ls="-", color="0.6", lw=1)

    axr.axvline(best_thresh, 0, 1, c="#353132", ls="--",
                label=r"$t_\mathrm{{opt}} = {:.2f}$".format(best_thresh))

    axr.plot(threshs[tw_id], pvals, c="0.3", label=r"KS-test $p$", lw=2.5)
    axr.plot(threshs[tw_id], scales, c="0.6", label="Scales", lw=2.5)

    axr.set_xlim(threshs[tw_id][0], threshs[tw_id][-1])
    axr.set_xlabel(r"Threshold $t$", fontsize=lblfntsize)
    axr.set_ylabel("Parameter value", fontsize=lblfntsize)
    axr.legend(loc="center left", fontsize=legfntsize)
#     axr.set_title(r"$t_\mathrm{{opt}} = {:.2f}$".format(best_thresh),
#                   fontsize=lblfntsize)
    axr.set_ylim(0, 2.5)

    return

In [ ]:
tw_chunks = [  # ID 20 goes extra
    [0, 1, 2],
    [3, 4, 5],
    [6, 7, 8],
    [9, 10, 11],
    [12, 13, 14],
    [15, 16, 17],
    [18, 19, 20],
]

x = np.linspace(0, max([bg_pdf_i.data.max() for
                        bg_pdf_i in bg_pdfs.values()]), 500)

try:
    # Almost full page ratio
    fs = latex_setup.make_figsize(ratio=0.8, scale=0.9)
    legfntsize = latex_setup.latex_settings["legend.fontsize"] - 3
    labfntsize = latex_setup.latex_settings["axes.labelsize"] - 3
except:
    fs = (5, 7.15)


for chunk in tw_chunks[:]:
    # Make a plot grid
    nrows, ncols = len(chunk), 2
    fig, axs = plt.subplots(nrows=nrows, ncols=ncols, figsize=fs,
                           sharex=False, sharey=False)
    for row, tw_id in enumerate(chunk):
        print("# Time window {}".format(tw_id))
        (axl, axr) = axs[row, 0], axs[row, 1]
        plot_row(axl=axl, axr=axr, res=res, threshs=threshs, tw_id=tw_id,
                 bg_pdfs=bg_pdfs, x=x)
        if row != nrows - 1:
            axl.set_xlabel("")
            axr.set_xlabel("")
        
    fig.tight_layout()
    save_fig(fig, "plots/time_dep/bg_ts_and_ks_tw_ids_" +
             "{}.pdf".format("_".join(["{}".format(d) for d in chunk])),
             bbox_inches="tight")
    plt.show()

Old standalone plots

In [ ]:
for tw_id in tw_ids:
    print("# Time window {}".format(tw_id))
    # pvalue guides
    plt.axhline(0.5, 0, 1, ls=":", color="0.6")
    plt.axhline(1, 0, 1, ls=":", color="0.6")

    best_thresh, best_idx, pvals, scales = res[tw_id]
    plt.axvline(best_thresh, 0, 1, c="#353132", ls="--",
                label=r"$t_\mathrm{{opt}} = {:.2f}$".format(best_thresh))

    plt.plot(threshs[tw_id], pvals, c="0.3", label=r"KS-test $p$", lw=2.5)
    plt.plot(threshs[tw_id], scales, c="0.6", label="Scales", lw=2.5)

    plt.xlim(threshs[tw_id][0], threshs[tw_id][-1])
    plt.xlabel(r"Threshold $t$")
    plt.ylabel("Parameter values")
    plt.title("Time window {}".format(tw_id + 1))
    plt.legend()

#     save_fig(plt.gcf(), "plots/bg_ts/bg_ts_thresh_scan_tw_{:2d}.pdf".format(tw_id),
#              bbox_inches="tight")
    plt.show()

Show bg only ts with exp tail

In [ ]:
x = np.linspace(0, max([bg_pdf_i.data.max() for
                        bg_pdf_i in bg_pdfs.values()]), 500)
c_hist, c_emp = "0.65", "#353132"

for tw_id in tw_ids:
    y = bg_pdfs[tw_id].pdf(x, dx=0.25)

    h, bins, err, norm = bg_pdfs[tw_id].data_hist(density=True, dx=0.25,
                                                  which="all")
    m = 0.5 * (bins[:-1] + bins[1:])
    
    # Clip manually for PGF backend
    ymin = np.amin(h[h>0] / 2.)
    h[h < ymin / 10.] = ymin / 10.
    y[y < ymin / 10.] = ymin / 10.

    plt.plot(bins, np.r_[h[0], h], drawstyle="steps-pre", color=c_hist)
    plt.errorbar(m, h, yerr=err, fmt=",", color=c_hist, zorder=-10)

    best_thresh = res[tw_id][0]
    plt.axvline(best_thresh, 0, 1, c=c_emp, ls="--")
    plt.plot(x, y, color=c_emp,
             label="Emp. PDF, $t_\mathrm{{opt}}={:.2f}$".format(best_thresh))

    plt.xlabel(r"Test statistic $-2\ln\Lambda$")
    plt.ylabel("PDF")
    plt.xlim(x[0], x[-1])
    plt.ylim(ymin, None)
    plt.yscale("log")
    plt.legend(loc="upper right")
    plt.title("Time window {}".format(tw_id + 1))

    save_fig(plt.gcf(), path="plots/bg_ts/bg_ts_tw_{}.pdf".format(tw_id),
             bbox_inches="tight")
    save_fig(plt.gcf(), path="plots/bg_ts/bg_ts_tw_{:2d}.pgf".format(tw_id),
             bbox_inches="tight")
    plt.show()

## KS cross check with lido trials

In [ ]:
%%time
bg_pdfs = loader.bg_pdf_loader(idx="all")
tw_ids = sorted(bg_pdfs.keys())

def _load_lido_bg_pdfs(tw_id):
    path = os.path.join(PATHS.local, "bg_pdfs_lido",
                        "bg_pdf_tw_{:02d}.json.gz".format(tw_id))
    with gzip.open(path) as json_file:
        emp_dist = tdu.ExpTailEmpiricalDist.from_json(json_file)
        
    return emp_dist

lido_pdfs = {tw_id: _load_lido_bg_pdfs(tw_id) for tw_id in tw_ids}

Scan best threshold for original trials.

In [ ]:
%%time
threshs = []
res = []

for tw_id in tw_ids:
    print("# Time window {}".format(tw_id))      
    # Scan in a range with still good statistics, but leave the really good
    # statistics part to the empirical PDF
    bg_pdfs[tw_id].fit_thresh(np.amax(bg_pdfs[tw_id].data))
    lo, hi = bg_pdfs[tw_id].ppf(q=100. * sigma2prob([3., 5.5]))
    threshs.append(np.arange(lo, hi, 0.1))
    # Best fit: KS test p-value is larger than `pval_thresh` the first time
    pval_thresh = 0.5
    res_i = tdu.scan_best_thresh(
        bg_pdfs[tw_id], threshs[-1], pval_thresh=pval_thresh)
    best_thresh, best_idx, pvals, scales = res_i
    res.append(res_i)

    best_thresh_sigma = prob2sigma(bg_pdfs[tw_id].cdf(best_thresh))
    print("Best thresh at TS = {:.2f}, {:.2f} sigma".format(
        best_thresh, best_thresh_sigma[0]))

With the best fit original BG PDFs, do a KS test with the exponential tail and the independent Lido trial tail data.

In [ ]:
all_ids = np.sort(bg_pdfs.keys())
pvals = []
for _tw in all_ids:
    # Get Lido tail data
    lido_pdfs[_tw].fit_thresh(bg_pdfs[_tw].thresh)
    lido_over_thresh_data, _ = lido_pdfs[_tw].get_split_data(emp=False)  
    # Do a KS test with the original exp tail and the Lido data
    loc, scale = bg_pdfs[_tw].thresh, bg_pdfs[_tw].scale
    pvals.append(scs.kstest(lido_over_thresh_data, "expon",
                            args=(loc, scale)).pvalue)
    print("Testing tw {}, pval is {:.2f}sigma".format(
        _tw, prob2sigma(1. - pvals[-1])[0]))

In [ ]:
# Plot KS pvalue vs. time window. No outliers seen, fit is stable.
# Only a bit fewer than 2/3 within 1sig and 19/20 within 2sig
ls = ["-", "-.", "--"]
for i, sigi in enumerate(sigma2prob([1, 2, 3])):
    plt.axhline(1. - sigi, 0, 1, ls=ls[i], c="C7",
                label=r"${:.0f}\sigma$".format(i + 1))

plt.plot(all_ids + 1, pvals, color="#353132", ls="", marker="o")

plt.xticks(all_ids[::2] + 1)
plt.yscale("log")

plt.xlabel("Time window ID")
plt.ylabel("KS test p-value")
plt.xlim(all_ids[0] + 0.5, all_ids[-1] + 1.5)
plt.ylim(1e-4, 1)
plt.legend(loc="lower left")
plt.tight_layout()

save_fig(plt.gcf(),"plots/time_dep/ks_test_lido_trials_pvals.pdf",
         bbox_inches="tight")
plt.show()

## HESE map source injection

In [ ]:
# Reconstruct a map from the stored CDF
key = "IC79"
src_idx = 0
sig_inj = multi_sig_inj.injs[key]
src_map = np.r_[0, np.diff(sig_inj._src_map_CDFs[src_idx])]
src_rec = sig_inj.srcs[src_idx]
src_dec = src_rec["dec"]
src_ra = src_rec["ra"]

In [ ]:
res = (500, 500)

def dec_to_theta(dec):
    return np.pi / 2. - np.array(dec)

# Zoom around best fit
dra, ddec = np.deg2rad(5), np.deg2rad(5)
bounds = [
    [src_ra - dra, src_ra + dra],
    dec_to_theta([src_dec - ddec, src_dec + ddec]),
]

src_map_ren, cx, cy = mplt.render_healpymap(src_map, npix=res, bounds=bounds,
                                            smooth=4.)
# Make -log10(PDF), so levels show sigma contours when approx. gaussian like
# m = (src_map > 0.)
src_map_ren = np.log10(src_map_ren)
src_map_ren = np.amax(src_map_ren) - src_map_ren

Sample some new src positions for this map

In [ ]:
nsamples = 1000
ras, decs = np.zeros(nsamples, dtype=float), np.zeros(nsamples, dtype=float)
for i in range(nsamples):
    sig_inj.sample(n_samples=1)
    ras[i] = sig_inj._src_ra[src_idx]
    decs[i] = sig_inj._src_dec[src_idx]

Plot map with sampled positions

In [ ]:
# Untruncated on the left truncated on the right
try:
    fs = latex_setup.make_figsize(scale=0.9, ratio=1.5)
except:
    fs = (5.2, 2.)
fig, ax = plt.subplots(1, 1, figsize=fs)

def ra_to_hours(ra):
    return 24. - mx / 360. * 24  # ra in deg -> ra in hours, right to left
mx, my = map(lambda b: np.rad2deg(0.5 * (b[:-1] + b[1:])), [cx, cy])
# Convert to equ. with ra from right to left in hours
# mx = 24. - mx / 360. * 24
my = 90. - my
xx, yy = np.meshgrid(mx, my)
levels = - scs.chi2.logsf(np.arange(0., 3, 0.5)**2, df=2)

img = ax.contourf(xx, yy, src_map_ren, levels, cmap="Greys_r", extend="max")
# ax.plot(np.rad2deg(src_ra), np.rad2deg(src_dec), c="w", ls="", marker="x")
ax.plot(np.rad2deg(ras), np.rad2deg(decs),
        marker=".", ls="", c="w", mec="#353132", lw=1, ms=6, alpha=0.75)

ax.set_xlabel(r"Right-ascension $\alpha$ in $^\circ$")
ax.set_ylabel(r"Declination $\delta$ in $^\circ$")
ax.grid(ls=":")

fig.tight_layout()
save_fig(fig, "plots/time_dep/hese_map_sampling.pdf", bbox_inches="tight", dpi=300)
plt.show()

## Differential performances

Load diff performance sets and bg pdf.

Note: Because the power law is fixed at E^-2, the E2 weighting, does not need to be explicitely done.
The flux norm at 1GeV is the same numerical value as the on at 100TeV weighted with 100TeV^2 (E^-2 is flat in E2 weighting).

In [ ]:
%%time
sig_inj_type = "healpy"
perf_trials = loader.perf_trials_loader(sig_inj_type, diff=True, idx="all")

bg_pdfs = loader.bg_pdf_loader("all")

In [ ]:
def weight_Egamma(enu, flux, gamma=2.):
    return enu**gamma * flux


def fluxatE0(flux, E0, E0new, gamma=2.):
    """ Base power law flux on a different reference energy. """
    return flux * (E0new / E0)**-gamma


def mu2flux(mu, mu2flux):
    """
    Converts mu to flux from the stored mu2flux(1) from the injector.
    Flux normalization in units from the injection, perf["E0"], perf["gamma"]
    in 1 / (GeV cm^2 s).
    """
    return np.atleast_1d(mu2flux) * np.atleast_1d(mu)


def guess_chi2_cdf_seed(mus, ts, ts_val):
    """
    When fitting a chi2 CDF to x=mus and y=cdfs, with df and loc (scale fixed,
    because it is degenerate to df), this finds a reasonable seed.
    From wikipedia (df = k) for larger k >> 2:
        Mean = k
        Median approx k (1 - 2 / (9k))^3
        Variance = 2k
    Also, loc = df for large k, because the median formula tends to k.
    This means, a resonable first guess proxy for the df and loc is
        df approx loc approx d**2 / 2
    where d is the distance from the 50% and 84% (or 16%) percentile as a proxy
    when assuming gaussian like behaviour for the stddevs.
    """
    # Get median mu
    idx_med = None
    for i, tsi in enumerate(ts):
        cdfi, _ = weighted_cdf(x=tsi, val=ts_val, weights=None)
        if cdfi < 0.5:
            idx_med = i
            break
    if idx_med is None:
        raise ValueError("Could not find median. " +
                         "Too few trials or ts val wrong?")
    # Get stddev mu (84%)
    idx_std = None
    for i, tsi in enumerate(ts):
        cdfi, _ = weighted_cdf(x=tsi, val=ts_val, weights=None)
        if cdfi < 0.16:
            idx_std = i
            break
    if idx_std is None:
        raise ValueError("Could not find 16% percentile. " +
                         "Too few trials or ts val wrong?")
    df = (mus[idx_std] - mus[idx_med])**2 / 2.
    return df, mus[idx_med] - df
    
    
def fit_all_chi2_params(mus, ts, ts_bf, beta):
    """
    Wrapper to fit the chi2 params for all bins.
    """
    out, seeds = [], []   
    for i in range(len(ts)):
        # tw 20 includes post processed trials with different mus.
        mus_ = mus[i]
        try:
            len(mus_)
        except:
            mus_ = mus
        # Guess a p0
        try:
            df, loc = guess_chi2_cdf_seed(mus_, ts[i], ts_bf)
        except ValueError:
            df, loc = 1, 1
        seeds.append([df, loc])
        # 90% UL (Needed signal for 90% over TS values of data result)
        out.append(fit_chi2_cdf(mus=mus_, ts=ts[i], beta=beta, ts_val=ts_bf,
                                p0=(df, loc, 1.), fscale=None))

    mu_bfs = np.array([o[0] for o in out])
    cdfs = np.array([o[1] for o in out])
    pars = np.array([o[2] for o in out])
    return mu_bfs, cdfs, pars


def plot_diff_limits(
    logE_bins, flux_bfs, ls=None, leg_labels=None, title=None, save_args=None):
    if plt.rcParams["text.usetex"]:
        labels = [
            r"$log_{10}(E_\nu\,/\,\si{\GeV})$",
            r"$E^2\phi_0$ at $\SI{100}{\TeV}$ in " + 
            r"$\si[per-mode=reciprocal]{\GeV\per\cm\squared\per\second}$"]
    else:
        labels = [
            r"$log_{10}(E_\nu\,/\,\mathrm{GeV})$",
            r"$E^2\phi_0$ at $100\,\mathrm{TeV}$ in " + 
            r"$\mathrm{GeV} / (\mathrm{cm^2\, s})$"]

    plt_leg = True
    if leg_labels is None:
        plt_leg = False
    # c = plt.cm.Greys_r(np.linspace(0., 0.3, len(flux_bfs)))
    keys = logE_bins.keys()
    c = {k: "#353121" for k in keys}
    if ls is None:
        ls = {k: "-" for k in keys}
    
    for key in keys:
        plt.plot(logE_bins[key], np.r_[flux_bfs[key][0], flux_bfs[key]],
                 c=c[key], ls=ls[key], drawstyle="steps-pre",
                 label=leg_labels[key])
#     plt.yscale("log")
    plt.xlabel(labels[0])
    plt.ylabel(labels[1])
    if title is not None:
        plt.title(title)
    plt.grid(ls=":")
    plt.xlim(2, 9)
    plt.ylim(0, None)
    if plt_leg:
        plt.legend()
        
    if save_args is not None:
        save_fig(plt.gcf(), **save_args)
        
    plt.show()

Calc the differential sensitivity and discover potential

In [ ]:
mu_bfs, cdfs, pars, flux_bfs = {}, {}, {}, {}

# Sensitity = avg. UL and discorvery potential
sigmas = {"sens": 0., "disc":  5.}
betas = {"sens": 0.9, "disc":  0.5}

# For each time window
tw_ids = sorted(bg_pdfs.keys())
# hotfix_id = [100 * 18 + 0]  # Combined ID where to apply a hotfix chi2 solution

for tw_id in tw_ids:
    print(tw_id)
    bg_pdf = bg_pdfs[tw_id]
    perf = perf_trials[tw_id]
    # Get the BG TS values for the desired betas
    mu_bfs[tw_id], cdfs[tw_id], pars[tw_id], flux_bfs[tw_id] = {}, {}, {}, {}
    for key in sigmas.keys():
        bg_ts_val = bg_pdf.ppf(q=100. * sigma2prob(sigmas[key]))
        _out = fit_all_chi2_params(
            perf["mus"], perf["ts"], bg_ts_val, betas[key])
        mu_bfs[tw_id][key], cdfs[tw_id][key], pars[tw_id][key] = _out
        flux_bfs[tw_id][key] = multi_sig_inj.mu2flux(mu_bfs[tw_id][key])

Make the plots

In [ ]:
if plt.rcParams["text.usetex"]:
    labels = [
        r"$\log_{10}(E_\nu\,/\,\si{\GeV})$",
        r"$E^2\Phi_0^{100\,\mathrm{TeV}}$",
    ]
else:
    labels = [
        r"$log_{10}(E_\nu\,/\,\mathrm{GeV})$",
        r"$E^2\Phi_0$ at $100\,\mathrm{TeV}$ in " + 
        r"$\mathrm{GeV}\,\mathrm{cm}^{-1}$"]
     
ls = {"sens": "-", "disc": "-"}
c = {"sens": "#353121", "disc": "0.5"}
leg_labels = {"sens": "Sensitivity", "disc": "Disc.\nPotential"}
# hotfix_id = [100 * 18 + 0]  # Combined ID where to apply a hotfix chi2 solution

# Plot in 2 (5x2) grids and a single one
try:
    titlesize = latex_setup.latex_settings["axes.labelsize"] - 5
    labsize = latex_setup.latex_settings["axes.labelsize"] - 5
    fs = latex_setup.make_figsize(scale=0.95, ratio=0.8)  # Full page
except:
    figsize = (5, 7.2)
nrows, ncols = 7, 3
fig, axs = plt.subplots(nrows, ncols, figsize=fs, sharex=True)
# Manual x and yticks and labels per row
xticks = range(2, 10, 1)
xticklabels = ["{:d}".format(xt) for xt in xticks]
yticks = [
    [0, 0.1, 0.2],
    [0, 0.1, 0.2],
    [0, 0.1, 0.2],
    [0, 0.1, 0.2, 0.3],
    [0, 0.1, 0.2, 0.3, 0.4, 0.5],
    [0, 0.5, 1],
    [0, 0.5, 1, 1.5, 2, 2.5],
]
yticks_minor = [
    [0.05, 0.15],
    [0.05, 0.15],
    [0.05, 0.15],
    None,
    None,
    [0.25, 0.75],
    None,
]
ymax = [yt[-1] for yt in yticks]
# Manually expend without altering the ticks
ymax[-2] = 1.25
ymax[-1] = 2.75

# Plot performance curves per time window
for tw_id in tw_ids[:]:
    bins = perf_trials[tw_id]["log_E_bins"]
    flbfs = flux_bfs[tw_id]
    row, col = idx2rowcol(idx=tw_id, ncols=ncols)
    ax = axs[row, col]

    for key in sigmas.keys():
        ax.plot(bins, np.r_[flbfs[key][0], flbfs[key]],
                c=c[key], ls=ls[key], drawstyle="steps-pre",
                label=leg_labels[key])

    ax.set_title("Time window {}".format(tw_id + 1), fontsize=titlesize)
    ax.grid(b=True, ls=":", which="major")
    ax.set_xlim(2, 9)
    # Ticklabels only at the bottom and left plots
    if row == nrows - 1:
        ax.set_xticks(xticks)
        ax.set_xticklabels(xticklabels)
        ax.set_xlabel(labels[0], fontsize=labsize)
    if col == 0:
        ax.set_ylabel(labels[1], fontsize=labsize)
    # For some yaxis set minor ticks with grid to have similar grey values
    if yticks_minor[row] is not None:
        ax.set_yticks(yticks_minor[row], minor=True)
        ax.grid(b=True, ls=":", which="minor")
    ax.set_yticks(yticks[row])
    ax.set_yticklabels(["{:.1f}".format(yt) for yt in yticks[row]],
                       fontsize=labsize)
    ax.set_ylim(0, ymax[row])

    # Put legend manually split in 2 last plots
    if (row == nrows - 1):
        _handles, _lables = ax.get_legend_handles_labels()
        if col == ncols - 2:
            ax.legend(_handles[:1], _lables[:1], loc="upper right",
                      prop={"size": labsize})
        if col == ncols - 1:
            ax.legend(_handles[1:], _lables[1:], loc="upper right",
                      prop={"size": labsize})

fig.tight_layout(w_pad=0., h_pad=0.25)
save_fig(fig, "plots/time_dep/diff_perf.pdf", bbox_inches="tight")
plt.show()

Output the values for a latex table

In [ ]:
from mypyscripts.general import arr2str

bins = perf_trials[0]["log_E_bins"]
s1 = slice(0, len(bins) // 2, 1)
s2 = slice(len(bins) // 2, len(bins) - 1, 1)

# First half of the table, slice 1
print("   & " + arr2str(bins[s1], fmt="{:.1f}  ", sep=" & ") + r" \\")
for tw_id in tw_ids:
    print("{:2d} & ".format(tw_id) +
          arr2str(flux_bfs[tw_id]["sens"][s1], fmt="{:.3f}", sep=" & ") + r" \\")
#     print("{:2d} & sens. & ".format(tw_id) +
#           arr2str(flux_bfs[tw_id]["sens"][s1], fmt="{:.3f}", sep=" & ") + r" \\")
#     print("   & disc. & " +
#           arr2str(flux_bfs[tw_id]["disc"][s1], fmt="{:.3f}", sep=" & ") + r" \\")
    
print("\n\n")
    
# Second half of the table in a new table, slice 2
print("   & " + arr2str(bins[s2], fmt="{:.1f}", sep=" & ") + r" \\")
for tw_id in tw_ids:
    print("{:2d} & ".format(tw_id) +
          arr2str(flux_bfs[tw_id]["sens"][s2], fmt="{:.3f}", sep=" & ") + r" \\")
#     print("{:2d} & sens. & ".format(tw_id) +
#           arr2str(flux_bfs[tw_id]["sens"][s2], fmt="{:.3f}", sep=" & ") + r" \\")
#     print("   & disc. & " +
#           arr2str(flux_bfs[tw_id]["disc"][s2], fmt="{:.3f}", sep=" & ") + r" \\")

### Plot chi2 fits per time window for a logE bins

In [ ]:
try:
    titlesize = latex_setup.latex_settings["axes.labelsize"] - 5
    labsize = latex_setup.latex_settings["axes.labelsize"] - 4
    fs = latex_setup.make_figsize(scale=0.95, ratio=0.8)  # Full page
except:
    figsize = (5, 7.2)

cs = {"sens": "#353132", "disc": "0.5"}
label = {"sens": "Sensitivity", "disc": "Discovery\nPotential"}
nrows, ncols = 5, 3

# For each time window
for tw_id in tw_ids[:]:
    bins = perf_trials[tw_id]["log_E_bins"]
    mus = perf_trials[tw_id]["mus"]
    # 5 x 3 Grid for each time window
    fig, axs = plt.subplots(nrows, ncols, figsize=fs, sharey=True)
    # For each energy bin
    for i in range(len(bins) - 1):
        row, col = idx2rowcol(idx=i, ncols=ncols)
        ax = axs[row, col]
        
        # tw 20 includes post processed trials with different mus.
        mus_ = mus[i]
        try:
            len(mus_)
        except:
            mus_ = mus         
        
        # Get xmax value from largest mu bf in both sens. and disc. pot.
        _mu_bf = {}
        for key in sigmas.keys():
            _mu_bf[key] = mu_bfs[tw_id][key][i]
        xmax = np.ceil(1.5 * max(list(_mu_bf.values())))
        x = np.linspace(0, xmax, 500)

        # For sens, disc
        for key in sigmas.keys():
            cdf_tw = cdfs[tw_id][key]
            pars_tw = pars[tw_id][key]
            y = scs.chi2.cdf(x, *pars_tw[i])
            
            ax.axhline(betas[key], 0, 1, ls="-", c="0.75")
            ax.axvline(_mu_bf[key], 0, 1, ls="-.", c=cs[key])
            
            ax.plot(x, y, c=cs[key], label=label[key], zorder=9)
            ax.plot(mus_, 1. - cdf_tw[i],
                    marker="o", c=cs[key], ls="", zorder=10, ms=3)
            
        title = (r"${:.1f}\leq \log_{{10}}".format(bins[i]) +
                 r"(E_\nu\,/\,\si{{GeV}}) < {:.1f}$".format(bins[i+1]))
        ax.text(x=0.5, y=1.025, s=title, fontsize=titlesize,
                va="bottom", ha="center", transform=ax.transAxes)
        ax.set_yticks([0, 0.5, 1])
        ax.set_yticklabels([r"$0$", r"$\frac{1}{2}$", r"$1$"])
        ax.set_xlim(0, xmax)
        ax.set_ylim(0, 1)
        if (row == nrows - 1) or ((row == nrows - 2) and (col == ncols - 1)):
            ax.set_xlabel(r"Signal evts. $\mu$", fontsize=labsize)
        if col == 0:
            ax.set_ylabel("CDF", fontsize=labsize)

    axs[-1, -1].remove()  # 14 bins, 15 axes...
    handles, labels = ax.get_legend_handles_labels()
    fig.legend(handles, labels, loc="center left", bbox_to_anchor=(0.7, 0.125))
    
    fig.tight_layout(h_pad=0, w_pad=0.5)
    save_fig(fig, "plots/time_dep/diff_perf_chi2_fits_tw_" +
             "{:02d}.pdf".format(tw_id), bbox_inches="tight")
    plt.show()

## Time win vs. performance

Note: Because the power law is fixed at E^-2, the E2 weighting, does not need to be explicitely done.
The flux norm at 1GeV is the same numerical value as the on at 100TeV weighted with 100TeV^2 (E^-2 is flat in E2 weighting).

In [ ]:
%%time
bg_pdfs = loader.bg_pdf_loader("all")
tw_ids = sorted(bg_pdfs.keys())
dt0s, dt1s = loader.time_window_loader(idx=tw_ids)
perf_trials = loader.perf_trials_loader(typ="healpy", idx="all", skylab=False)

In [ ]:
def weight_Egamma(enu, flux, gamma=2.):
    return enu**gamma * flux

def fluxatE0(flux, E0, E0new, gamma=2.):
    """ Base power law flux on a different reference energy. """
    return flux * (E0new / E0)**-gamma

def guess_chi2_cdf_seed(mus, ts, ts_val):
    """
    When fitting a chi2 CDF to x=mus and y=cdfs, with df and loc (scale fixed,
    because it is degenerate to df), this finds a reasonable seed.
    From wikipedia (df = k) for larger k >> 2:
        Mean = k
        Median approx k (1 - 2 / (9k))^3
        Variance = 2k
    Also, loc = df for large k, because the median formula tends to k.
    This means, a resonable first guess proxy for the df and loc is
        df approx loc approx d**2 / 2
    where d is the distance from the 50% and 84% (or 16%) percentile as a proxy
    when assuming gaussian like behaviour for the stddevs.
    """
    # Get median mu
    idx_med = None
    for i, tsi in enumerate(ts):
        cdfi, _ = weighted_cdf(x=tsi, val=ts_val, weights=None)
        if cdfi < 0.5:
            idx_med = i
            break
    if idx_med is None:
        raise ValueError("Could not find median. " +
                         "Too few trials or ts val wrong?")
    # Get stddev mu (84%)
    idx_std = None
    for i, tsi in enumerate(ts):
        cdfi, _ = weighted_cdf(x=tsi, val=ts_val, weights=None)
        if cdfi < 0.16:
            idx_std = i
            break
    if idx_std is None:
        raise ValueError("Could not find 16% percentile. " +
                         "Too few trials or ts val wrong?")
    df = (mus[idx_std] - mus[idx_med])**2 / 2.
    return df, mus[idx_med] - df

Precalc values

In [ ]:
# Sensitity = avg. UL and discorvery potential
sigmas = {"sens": 0., "disc":  5.}
betas = {"sens": 0.9, "disc":  0.5}

# For each time window store the stats
mu_bfs, cdfs, pars, flux_bfs = {}, {}, {}, {}
for tw_id in tw_ids:
    print(tw_id)
    bg_pdf = bg_pdfs[tw_id]
    perf = perf_trials[tw_id]
    # Get the BG TS values for the desired betas
    mu_bfs[tw_id], cdfs[tw_id], pars[tw_id], flux_bfs[tw_id] = {}, {}, {}, {}
    for key in sigmas.keys():
        bg_ts_val = bg_pdf.ppf(q=100. * sigma2prob(sigmas[key]))
        try:
            df, loc = guess_chi2_cdf_seed(perf["mus"], perf["ts"], bg_ts_val)
        except ValueError:
            df, loc = 1, 1
        # 90% UL (Needed signal for 90% over TS values of data result)
        _out = fit_chi2_cdf(mus=perf["mus"], ts=perf["ts"], beta=betas[key],
                            ts_val=bg_ts_val,
                            p0=(df, loc, 1.), fscale=None)

        mu_bfs[tw_id][key], cdfs[tw_id][key], pars[tw_id][key] = _out
        flux_bfs[tw_id][key] = multi_sig_inj.mu2flux(mu_bfs[tw_id][key])

Make the time windows vs. performance plots

In [ ]:
# Poisson 90% UL: For which expectation mu are 90% over n=0 events?
beta = 0.9
a, b = 2, 3  # It's somewhere around 2.3
pois_ul = sco.brentq(lambda mu: scs.poisson.sf(0, mu) - beta, a, b)
pois_ul_flux = multi_sig_inj.mu2flux(pois_ul)

In [ ]:
delts = dt1s - dt0s

sens = [v["sens"] for v in flux_bfs.values()]
disc = [v["disc"] for v in flux_bfs.values()]

fs = latex_setup.make_figsize()
fig, ax  = plt.subplots(1, 1, figsize=fs)

ax.axhline(pois_ul_flux, 0, 1, ls="--", c="#353132", lw=1,
           label="90\% Poisson UL")
ax.plot(delts, sens, c="#353132", lw=2.5, label="Sensitivity")
ax.plot(delts, disc, c="0.5", lw=2.5, label="Discovery\nPotential")

ax.set_xscale("log")
ax.set_xlim(delts[0], delts[-1])
ax.set_xlabel(r"Time window length in \si{\second}")
ax.set_ylabel(r"$E^2\Phi_0$ at $\SI{100}{TeV}$ in " + 
              r"$\si[per-mode=reciprocal]{\GeV\per\cm\squared}$")
ax.legend(loc="upper left")

fig.tight_layout()
save_fig(fig, "plots/time_dep/perf.pdf", bbox_inches="tight")
plt.show()

### Chi2 fits

Make the plot grid

In [ ]:
ls = {"sens": "-", "disc": "-"}
c = {"sens": "#353121", "disc": "0.5"}
leg_labels = {"sens": "Sensitivity", "disc": "Disc.\nPotential"}

try:
    labsize = latex_setup.latex_settings["axes.labelsize"] - 5
    fs = latex_setup.make_figsize(scale=0.95, ratio=0.8)  # Full page
except:
    figsize = (5, 7.2)
nrows, ncols = 7, 3
fig, axs = plt.subplots(nrows, ncols, figsize=fs, sharey=True)

# Plot chi2 fit curves per time window
for tw_id in tw_ids[:]:
    mus = perf_trials[tw_id]["mus"]
    mu_bfs_tw = mu_bfs[tw_id]
    flbfs = flux_bfs[tw_id]
    cdf_tw = cdfs[tw_id]
    pars_tw = pars[tw_id]

    row, col = idx2rowcol(idx=tw_id, ncols=ncols)
    ax = axs[row, col]
        
    # Get xmax value from largest mu bf in both sens. and disc. pot.
    _mu_bf = {}
    for key in sigmas.keys():
        _mu_bf[key] = mu_bfs_tw[key]
    xmax = np.ceil(1.5 * max(list(_mu_bf.values())))
    x = np.linspace(0, xmax, 500)

    # For sens, disc
    for key in sigmas.keys():
        y = scs.chi2.cdf(x, *pars_tw[key])

        ax.axhline(betas[key], 0, 1, ls="-", c="0.75")
        ax.axvline(_mu_bf[key], 0, 1, ls="-.", c=c[key])

        ax.plot(x, y, c=c[key], label=leg_labels[key], zorder=9)
        ax.plot(mus, 1. - cdf_tw[key],
                marker="o", c=c[key], ls="", zorder=10, ms=3)
    title = r"Time window {:d}".format(tw_id + 1)
    ax.text(x=0.5, y=1.05, s=title, fontsize=labsize + 1,
            va="bottom", ha="center", transform=ax.transAxes)
    ax.set_yticks([0, 0.5, 1])
    ax.set_yticklabels([r"$0$", r"$\frac{1}{2}$", r"$1$"])
    ax.set_xlim(0, xmax)
    ax.set_ylim(0, 1)
    if (row == nrows - 1):
        ax.set_xlabel(r"Signal evts. $\mu$", fontsize=labsize + 1)
    if col == 0:
        ax.set_ylabel("CDF", fontsize=labsize + 1)
    
    # Put legend manually split in 2 last plots
    if (row == nrows - 1):
        _handles, _lables = ax.get_legend_handles_labels()
        if col == ncols - 2:
            ax.legend(_handles[:1], _lables[:1], loc="lower right",
                      prop={"size": labsize})
        if col == ncols - 1:
            ax.legend(_handles[1:], _lables[1:], loc="lower right",
                      prop={"size": labsize})

fig.tight_layout(h_pad=0.25, w_pad=0.25)
save_fig(fig, "plots/time_dep/perf_chi2_fits.pdf", bbox_inches="tight")
plt.show()

## Show exp. result pre and post trial

In [ ]:
with open(os.path.join(PATHS.local, "unblinding", "result.json")) as fp:
    result = json.load(fp)
    
tw_id = 20
bg_pdf = loader.bg_pdf_loader(tw_id)[tw_id]

fname = "data/post_neglog10_pdf.json"
with open(fname) as fp:
    post_neglog10_pdf = tdu.stats.PureEmpiricalDist.from_json(fp=fp)
    print("Loaded post trials from:\n  {}".format(fname))

In [ ]:
c_hist, c_emp = "0.65", "#353132"
legsize = latex_setup.latex_settings["legend.fontsize"] - 3
fs = latex_setup.make_figsize(ratio=2.5)

# Left BG TS with exp result, right post trial PDF with exp result
fig, (axl, axr) = plt.subplots(1, 2, figsize=fs)

## Left BG TS
x = np.linspace(0, round_mantissa(np.amax(bg_pdf.data)), 500)
y = bg_pdf.pdf(x, dx=0.25)
h, bins, err, norm = bg_pdf.data_hist(density=True, dx=0.25, which="all")
m = 0.5 * (bins[:-1] + bins[1:])

# Clip manually for PGF backend
ymin = np.amin(h[h>0] / 2.)
h[h < ymin / 10.] = ymin / 10.
y[y < ymin / 10.] = ymin / 10.

axl.plot(bins, np.r_[h[0], h], drawstyle="steps-pre", color=c_hist)
axl.errorbar(m, h, yerr=err, fmt=",", color=c_hist, zorder=-10, lw=1)
axl.plot(x, y, color=c_emp)
axl.axvline(result["ts"][tw_id], 0, 1, c=c_emp, ls="--",
            label="Experimental result:\n" +
            r"$-2\ln\hat{{\Lambda}}={:.2f}$".format(result["ts"][tw_id]))

axl.set_xlabel(r"Test statistic $-2\ln\Lambda$")
axl.set_ylabel("PDF")
axl.set_title("Time window {}".format(tw_id + 1), fontsize=legsize + 1)
axl.set_xlim(x[0], x[-1])
axl.set_ylim(ymin, 3)
axl.set_yscale("log", nonposy="clip")
axl.legend(loc="upper right", prop={"size": legsize})

## Right post trial TS
dx = 0.1
bins = np.arange(0, 8 + dx, dx)
h, _ = np.histogram(post_neglog10_pdf.data, bins=bins, density=False)
err = np.sqrt(h)
norm = np.sum(h) * np.diff(bins)
h = h / norm
err = err / norm
m = 0.5 * (bins[:-1] + bins[1:])
pval = result["pvals"][tw_id]
sigma = prob2sigma(1. - pval)[0]
pval = -np.log10(pval)

axr.plot(bins, np.r_[h[0], h], drawstyle="steps-pre", color=c_emp)
axr.errorbar(m, h, yerr=err, fmt=",", color=c_emp, zorder=-10, lw=1)
axr.axvline(pval, 0, 1, c=c_emp, ls="--",
            label="Pre-trial $p={:.2f}\sigma$".format(sigma))
axr.set_xlabel(r"Pre-trial $-\log_{10}(p)$")
# axr.set_ylabel("PDF ")
axr.set_yscale("log")
axr.set_xlim(0, 8)
axr.set_xticks(range(0, 9))
axr.set_ylim(1e-5, 5e0)
axr.legend(loc="upper right", prop={"size": legsize})

fig.tight_layout()
save_fig(fig, path="plots/time_dep/exp_res_bg_pdf_post_pdf.pdf",
         bbox_inches="tight")
plt.show()

### Post trial and post trial <-> pre trial dependency

In [ ]:
c_emp = "#353132"
legsize = latex_setup.latex_settings["legend.fontsize"] - 3
fs = latex_setup.make_figsize(ratio=2.5)
pdata = post_neglog10_pdf.data

# Left BG TS with exp result, right post trial PDF with exp result
fig, (axl, axr) = plt.subplots(1, 2, figsize=fs)

## Left post trial
dx = 0.1
bins = np.arange(0, 8 + dx, dx)
h, _ = np.histogram(pdata, bins=bins, density=False)
err = np.sqrt(h)
norm = np.sum(h) * np.diff(bins)
h = h / norm
err = err / norm
m = 0.5 * (bins[:-1] + bins[1:])
sigmas = np.arange(1, 4)
pvals = post_neglog10_pdf.ppf(100. * sigma2prob(sigmas))

axl.plot(bins, np.r_[h[0], h], drawstyle="steps-pre", color=c_emp)
axl.errorbar(m, h, yerr=err, fmt=",", color=c_emp, zorder=-10, lw=1)
lss = ["-", "-.", "--"]
for j, pval in enumerate(pvals):
    axl.axvline(pval, 0, 1, c=c_emp, ls=lss[j], lw=1.,
                label=r"${:.0f}\sigma$".format(sigmas[j]))
axl.set_xlabel(r"Pre-trial $-\log_{10}(p)$")
axl.set_ylabel("PDF ")
axl.set_yscale("log")
axl.set_xlim(0, 8)
axl.set_xticks(range(0, 9))
axl.set_ylim(1e-5, 5e0)
axl.legend(loc="upper right", prop={"size": legsize})

## Right post trial <-> pre trial
# Range. Under threshold, no pvals are defined -> zero trials
thresh = np.amin(pdata[pdata > 0])
neglog10_pre_trial = np.linspace(thresh, -np.log10(1. - sigma2prob(4)), 200)
_post_trial_vals = post_neglog10_pdf.sf(neglog10_pre_trial)
neglog10_post_trial = -np.log10(_post_trial_vals)

axr.plot(prob2sigma(1. - 10**-neglog10_pre_trial),
         prob2sigma(1. - 10**-neglog10_post_trial), color=c_emp, lw=2)
axr.plot([0, 4], [0, 4], c="C7", ls="--", lw=1)
axr.set_xlabel(r"Pre-trial $\sigma$")
axr.set_ylabel(r"Post-trial $\sigma$")
axr.set_xlim(0, 4)
axr.set_ylim(0, 4)
axr.grid(ls=":")

fig.tight_layout()
save_fig(fig, path="plots/time_dep/post_trial_pre_trial.pdf",
         bbox_inches="tight")
plt.show()

# Discussion: time dependent plots

## Neyman plane

In [ ]:
%%time
path = os.path.join(PATHS.local, "unblinding", "result.json")
with open(path) as fp:
    result = json.load(fp)
    
_tw = 20
perf = loader.perf_trials_loader(typ="healpy", idx=_tw, skylab=False)[_tw]

In [ ]:
mus = perf["mus"]
ts = np.array(perf["ts"])

ts_bins = np.arange(0, np.ceil(np.amax(ts)), 0.25)

# Rows have mus, cols have ts, as if matrix is viewed as an image
hist = np.empty((len(mus), len(ts_bins) - 1), dtype=float)
for i in range(len(mus)):
    hist[i], _ = np.histogram(ts[i], bins=ts_bins, density=True)
    
# Create empirical Neyman bands
band_lo_emp = np.percentile(a=ts, axis=1, q=84)
band_hi_emp = np.percentile(a=ts, axis=1, q=16)
uls_emp = np.percentile(a=ts, axis=1, q=10)

In [ ]:
# Create mu borders for pcolormesh
dmu = np.diff(mus)
mu_mids = 0.5 * (mus[:-1] + mus[1:])
mu_borders = np.r_[mus[0] - dmu[0], mu_mids, mus[-1] + dmu[-1]]

XX, YY = np.meshgrid(ts_bins, mu_borders)
ZZ = np.array(hist)

In [ ]:
fs = latex_setup.make_figsize()
fig, ax = plt.subplots(1, 1, figsize=fs)
c_ln = "#353132"

# mesh = plt.pcolormesh(XX, YY, ZZ, cmap="Greys", norm=LogNorm())
# mesh.set_rasterized(True)
step = 0.25
levels = np.arange(-3, 0 + step, step)
vals = np.log10(np.pad(ZZ, pad_width=[(0, 1), (0, 1)], mode="edge"))

# Smooth along x, except for the first two bins, in which the zero trials are!
from scipy.ndimage.filters import gaussian_filter
vals[:, 2:] = gaussian_filter(vals[:, 2:], sigma=(0., 1.5))

img = ax.contourf(XX, YY, vals, levels, cmap="Greys", extend="both")
cbar = fig.colorbar(img, ax=ax)
cbar.set_label(r"$\log_{10}(\mathrm{PDF})$")

ax.plot(uls_emp, mus, c=c_ln, ls="--", label=r"\SI{90}{\percent} UL")
ax.plot(band_lo_emp, mus, c=c_ln, ls=":",
         label=r"\SI{68}{\percent} CL Band")
ax.plot(band_hi_emp, mus, c=c_ln, ls=":")

exp_ts = result["ts"][_tw]
ax.axvline(exp_ts, 0, 1, ls="-", c=c_ln, label="Exp. result")

ax.set_xlabel(r"Test statistic $-2\ln\Lambda$")
ax.set_ylabel(r"Mean no. of signal events $\mu$")

ax.set_xlim(0, 20)
ax.set_ylim(0, 30)
leg = plt.legend(loc="best")
# leg.get_frame().set_facecolor("0.65")
# plt.title(r"Empiric Neyman plane")

save_fig(fig, "plots/discussion_tdep/neyman_plane.pdf", bbox_inches="tight")
plt.show()

## UL per time window with sensitivity and "diffuse fluence"

Note: Because the power law is fixed at E^-2, the E2 weighting, does not need to be explicitely done.
The flux norm at 1GeV is the same numerical value as the on at 100TeV weighted with 100TeV^2 (E^-2 is flat in E2 weighting).

In [ ]:
%%time
bg_pdfs = loader.bg_pdf_loader("all")
tw_ids = sorted(bg_pdfs.keys())
dt0s, dt1s = loader.time_window_loader(idx=tw_ids)
perf_trials = loader.perf_trials_loader(typ="healpy", idx="all", skylab=False)

# Load unblided result
path = os.path.join(PATHS.local, "unblinding", "result.json")
with open(path) as fp:
    result = json.load(fp)

In [ ]:
def weight_Egamma(enu, flux, gamma=2.):
    return enu**gamma * flux

def fluxatE0(flux, E0, E0new, gamma=2.):
    """ Base power law flux on a different reference energy. """
    return flux * (E0new / E0)**-gamma

def guess_chi2_cdf_seed(mus, ts, ts_val):
    """
    When fitting a chi2 CDF to x=mus and y=cdfs, with df and loc (scale fixed,
    because it is degenerate to df), this finds a reasonable seed.
    From wikipedia (df = k) for larger k >> 2:
        Mean = k
        Median approx k (1 - 2 / (9k))^3
        Variance = 2k
    Also, loc = df for large k, because the median formula tends to k.
    This means, a resonable first guess proxy for the df and loc is
        df approx loc approx d**2 / 2
    where d is the distance from the 50% and 84% (or 16%) percentile as a proxy
    when assuming gaussian like behaviour for the stddevs.
    """
    # Get median mu
    idx_med = None
    for i, tsi in enumerate(ts):
        cdfi, _ = weighted_cdf(x=tsi, val=ts_val, weights=None)
        if cdfi < 0.5:
            idx_med = i
            break
    if idx_med is None:
        raise ValueError("Could not find median. " +
                         "Too few trials or ts val wrong?")
    # Get stddev mu (84%)
    idx_std = None
    for i, tsi in enumerate(ts):
        cdfi, _ = weighted_cdf(x=tsi, val=ts_val, weights=None)
        if cdfi < 0.16:
            idx_std = i
            break
    if idx_std is None:
        raise ValueError("Could not find 16% percentile. " +
                         "Too few trials or ts val wrong?")
    df = (mus[idx_std] - mus[idx_med])**2 / 2.
    return df, mus[idx_med] - df

Precalc values

In [ ]:
# Sensitity = avg. UL and discorvery potential
result["pvals"] = np.array(result["pvals"])
exp_sigmas = prob2sigma(1. - result["pvals"])
sigmas = {"sens": len(tw_ids) * [0.],
          "disc": len(tw_ids) * [5.],
          "lim":  exp_sigmas}
betas = {"sens": 0.9, "disc": 0.5, "lim":  0.9}

# For each time window store the stats
mu_bfs, cdfs, pars, flux_bfs = {}, {}, {}, {}
for tw_id in tw_ids:
    print(tw_id)
    bg_pdf = bg_pdfs[tw_id]
    perf = perf_trials[tw_id]
    # Get the BG TS values for the desired betas
    mu_bfs[tw_id], cdfs[tw_id], pars[tw_id], flux_bfs[tw_id] = {}, {}, {}, {}
    for key in sigmas.keys():
        bg_ts_val = bg_pdf.ppf(q=100. * sigma2prob(sigmas[key][tw_id]))
        try:
            df, loc = guess_chi2_cdf_seed(perf["mus"], perf["ts"], bg_ts_val)
        except ValueError:
            df, loc = 1, 1
        # 90% UL (Needed signal for 90% over TS values of data result)
        _out = fit_chi2_cdf(mus=perf["mus"], ts=perf["ts"],
                            beta=betas[key], ts_val=bg_ts_val,
                            p0=(df, loc, 1.), fscale=None)

        mu_bfs[tw_id][key], cdfs[tw_id][key], pars[tw_id][key] = _out
        flux_bfs[tw_id][key] = multi_sig_inj.mu2flux(mu_bfs[tw_id][key])

Make the time windows vs. performance plots

In [ ]:
# Diffuse flux to fluence curve
def diff_flux(E):
    """ Eight year diffuse fit, ICRC 2017 """
    phi0 = 1e-18  # In 1 / (GeV cm^2 sr s)
    gamma = 2.19
    E0 = 1e5      # 100 TeV
    return phi0 * (E / E0)**-gamma

# Tracks + Cascades, most optimistic case
Nhese = 82
# Optimistic 6 years full livetime to seconds
livetime = 6. * 365.25 * 24. * 60. * 60
# Transient HESE-like events per second per full sky
density = Nhese / (4. * np.pi * livetime)  # 1 / (sr s)

# Transform flux to fluence per source by integrating the flux over the livetime
# and the full sky and dividing by the number of HESE events.
# The time window size cancels in the integration and in the multiplication for
# the density (4pi * Tw * phi_diff) / (dens * 4pi * Tw)
diff_fluence_per_src = diff_flux(1e5) / density
diff_fluence = diff_fluence_per_src * Nhese

In [ ]:
delts = dt1s - dt0s
log_delts = np.log10(delts)
mids = 10**(0.5 * (log_delts[:-1] + log_delts[1:]))
xerr_lo = np.r_[delts[0], mids] - delts
xerr_hi = np.r_[mids, 2 * delts[-1]] - delts

Nhese_tracks = 22
E2w = (1e5)**2

sens = np.array([v["sens"] for v in flux_bfs.values()])   # / Nhese_tracks
disc = np.array([v["disc"] for v in flux_bfs.values()])   # / Nhese_tracks
lim = np.array([v["lim"] for v in flux_bfs.values()])     # / Nhese_tracks

fs = latex_setup.make_figsize()
fig, ax  = plt.subplots(1, 1, figsize=fs)

ax.axhline(diff_fluence * E2w, 0, 1, ls="-", c="#353132", lw=1.5,
           label="Diffuse fluence\nfor $82$ events")
ax.errorbar(10**np.arange(1, 6), 5 * [diff_fluence * E2w], fmt=",",
            yerr=diff_fluence * E2w / 5., uplims=True,
            c="#353132", lw=1.5, zorder=1)
# ax.plot(delts, lim, c="#353132", label="Exp. 90\% UL", marker="d", ls="",
#         zorder=10)
ax.errorbar(delts, lim, uplims=True, yerr=lim / 5.,
            xerr=[-0.8*xerr_lo, 0.8*xerr_hi],
            lw=2.5, c="#353132", fmt="_", ls="",
            label="Exp. 90\% UL", zorder=10)
ax.plot(delts, sens, c="0.5", lw=2.5, label="Sensitivity", ls="-")
ax.plot(delts, disc, c="0.5", lw=2.5, label="Discovery\nPotential", ls="--")

ax.set_xscale("log")
ax.set_yscale("log")
ax.set_xlim(delts[0], delts[-1] * 1.1)
ax.set_xlabel(r"Time window length in \si{\second}")
ax.set_ylabel(r"$E^2\Phi_0$ at $\SI{100}{TeV}$ in " + 
              r"$\si[per-mode=reciprocal]{\GeV\per\cm\squared}$")
ax.legend(loc="center left", ncol=2)
ax.grid(ls=":")
ax.set_axisbelow(True)

fig.tight_layout()
save_fig(fig, "plots/discussion_tdep/limits_vs_tw.pdf", bbox_inches="tight")
plt.show()

### Chi2 fits

Make the plot grid

In [ ]:
ls = {"lim": "-"}
c = {"lim": "#353121"}
leg_labels = {"lim": "Exp. 90\% UL"}

try:
    labsize = latex_setup.latex_settings["axes.labelsize"] - 5
    fs = latex_setup.make_figsize(scale=0.95, ratio=0.8)  # Full page
except:
    figsize = (5, 7.2)
nrows, ncols = 7, 3
fig, axs = plt.subplots(nrows, ncols, figsize=fs, sharey=True)

# Plot chi2 fit curves per time window
for tw_id in tw_ids[:]:
    mus = perf_trials[tw_id]["mus"]
    mu_bfs_tw = mu_bfs[tw_id]
    flbfs = flux_bfs[tw_id]
    cdf_tw = cdfs[tw_id]
    pars_tw = pars[tw_id]

    row, col = idx2rowcol(idx=tw_id, ncols=ncols)
    ax = axs[row, col]
        
    # Get xmax value from largest mu bf in both sens. and disc. pot.
    _mu_bf = {}
    for key in leg_labels.keys():
        _mu_bf[key] = mu_bfs_tw[key]
    xmax = np.ceil(1.5 * max(list(_mu_bf.values())))
    x = np.linspace(0, xmax, 500)

    # For sens, disc
    for key in leg_labels.keys():
        y = scs.chi2.cdf(x, *pars_tw[key])

        ax.axhline(betas[key], 0, 1, ls="-", c="0.75")
        ax.axvline(_mu_bf[key], 0, 1, ls="-.", c=c[key])

        ax.plot(x, y, c=c[key], label=leg_labels[key], zorder=9)
        ax.plot(mus, 1. - cdf_tw[key],
                marker="o", c=c[key], ls="", zorder=10, ms=3)
    title = r"Time window {:d}".format(tw_id + 1)
    ax.text(x=0.5, y=1.05, s=title, fontsize=labsize + 1,
            va="bottom", ha="center", transform=ax.transAxes)
    ax.set_yticks([0, 0.5, 1])
    ax.set_yticklabels([r"$0$", r"$\frac{1}{2}$", r"$1$"])
    ax.set_xlim(0, xmax)
    ax.set_ylim(0, 1)
    if (row == nrows - 1):
        ax.set_xlabel(r"Signal evts. $\mu$", fontsize=labsize + 1)
    if col == 0:
        ax.set_ylabel("CDF", fontsize=labsize + 1)
    
fig.tight_layout(h_pad=0.25, w_pad=0.25)
save_fig(fig, "plots/discussion_tdep/limits_vs_tw_chi2_fits.pdf",
         bbox_inches="tight")
plt.show()

## UL scan in gamma

Trials were injected for multiple gammas and time windows but always tested against the best fit model.

Because the largest time window was the best fit one, the curves doesn't vary with time window length, because when injecting in a smaller window but testing with a larger one, doesn't worsen the performance, because no additional separation power comes from the uniform time windows.

Here, a plot phi0 UL at 100 TeV vs. gamma is done, always for the largest time windows, as the time dependency is rather uninformative...

In [ ]:
%%time
# ######################################################
# Only load for tw_id = 20 (largest time window)
tw_id = 20
dt0, dt1 = loader.time_window_loader(idx=tw_id)
dt = dt1 - dt0

# Load gamma scan global trials
gamma_trials = []
path = os.path.join(PATHS.data, "limits_healpy")
files = glob(os.path.join(path, "tw_20_gamma_*.json.gz"))
for f in sorted(files):
    with gzip.open(f) as fp:
        perf = json.load(fp)
    gamma_trials.append(perf)

# Prepare gamma limits for calculations
gammas = np.array([p["gamma"] for p in gamma_trials])
flux_bfs = np.array([p["flux_bf"] for p in gamma_trials])  # At 1GeV
idx = np.argsort(gammas)
gammas = gammas[idx]
flux_bfs = flux_bfs[idx]
    
# #######################################################
# Load differential trials to show most sensitive regions per gamma
sig_inj_type = "healpy"
diff_trials = loader.perf_trials_loader(
    sig_inj_type, diff=True, idx=tw_id)[tw_id]

# BG pdfs and result for the ts value on data
bg_pdf = loader.bg_pdf_loader(tw_id)[tw_id]
with open(os.path.join(PATHS.local, "unblinding", "result.json")) as fp:
    result = json.load(fp)
    
print(":: Done ::")

In [ ]:
def round_to_decade(f, way="ceil"):
    if way == "ceil":
        return 10**(np.ceil(np.log10(f)))
    elif way == "floor":
        return 10**(np.floor(np.log10(f)))
    else:
        raise ValueError("'way")
        
def weight_Egamma(enu, flux, gamma=2.):
    return enu**gamma * flux

def fluxatE0(flux, E0, E0new, gamma=2.):
    """ Base power law flux on a different reference energy. """
    return flux * (E0new / E0)**-gamma

def fit_all_chi2_params(mus, ts, ts_bf, beta, seed_override_dict=None):
    """
    Wrapper to fit the chi2 params for all bins.
    """
    out, seeds = [], []   
    for i in range(len(ts)):
        # Per default, fit df, loc and scale with auto derived seeds.
        # Can be overridden with seed_override_dict[i] per bin
        fscale = None
        # tw 20 includes post processed trials with different mus.
        mus_ = mus[i]
        try:
            len(mus_)
        except:
            mus_ = mus
        # Guess a p0 if no override was given
        try:
            seed, fscale = seed_override_dict[i]
            print("Seed override for i={}:\n  ".format(i) +
                  "{}, fscale: {}".format(seed, fscale))
        except:
            try:
                df, loc = guess_chi2_cdf_seed(mus_, ts[i], ts_bf)
            except ValueError:
                df, loc = 1, 1
            seed = (df, loc, 1.)
        # Format: _out = [best fit mu, [cdf vals], [fitted chi2 pars])
        _out = fit_chi2_cdf(mus=mus_, ts=ts[i], beta=beta, ts_val=ts_bf,
                            p0=seed, fscale=fscale)
        # Manually append fscale if used
        if fscale is not None:
            _out = (_out[0], _out[1], [_out[2][0], _out[2][1], fscale])
        out.append(_out)
        # seeds.append(seed)

    mu_bfs = np.array([o[0] for o in out])
    cdfs = np.array([o[1] for o in out])
    pars = np.array([o[2] for o in out])
    return mu_bfs, cdfs, pars

def guess_chi2_cdf_seed(mus, ts, ts_val):
    """
    When fitting a chi2 CDF to x=mus and y=cdfs, with df and loc (scale fixed,
    because it is degenerate to df), this finds a reasonable seed.
    From wikipedia (df = k) for larger k >> 2:
        Mean = k
        Median approx k (1 - 2 / (9k))^3
        Variance = 2k
    Also, loc = df for large k, because the median formula tends to k.
    This means, a resonable first guess proxy for the df and loc is
        df approx loc approx d**2 / 2
    where d is the distance from the 50% and 84% (or 16%) percentile as a proxy
    when assuming gaussian like behaviour for the stddevs.
    """
    # Get median mu
    idx_med = None
    for i, tsi in enumerate(ts):
        cdfi, _ = weighted_cdf(x=tsi, val=ts_val, weights=None)
        if cdfi < 0.5:
            idx_med = i
            break
    if idx_med is None:
        raise ValueError("Could not find median. " +
                         "Too few trials or ts val wrong?")
    # Get stddev mu (84%)
    idx_std = None
    for i, tsi in enumerate(ts):
        cdfi, _ = weighted_cdf(x=tsi, val=ts_val, weights=None)
        if cdfi < 0.16:
            idx_std = i
            break
    if idx_std is None:
        raise ValueError("Could not find 16% percentile. " +
                         "Too few trials or ts val wrong?")
    df = (mus[idx_std] - mus[idx_med])**2 / 2.
    return df, mus[idx_med] - df

def power_law_integral(logE_bins, gamma, phi0, E0):
    """ int_a^b f(E) dE with f(E) = phi0 * (E / E0)**-gamma """
    Elo, Ehi = 10**logE_bins[:-1], 10**logE_bins[1:]
    if gamma == 1.:
        res = phi0 * E0 * (np.log10(Ehi / E0) - np.log10(Elo / E0))
    else:
        res = phi0 / (gamma - 1.)
        res *= (Elo * (Elo / E0)**-gamma - Ehi * (Ehi / E0)**-gamma)
    return res

def get_power_law_ints(logE_bins, phi0s, E0s, gammas):
    ints = []
    for i in range(len(logE_bins) - 1):
        ints.append(power_law_integral(E0=E0s[i], gamma=gammas[i],
                                       logE_bins=logE_bins, phi0=phi0s[i]))
    return ints

def get_global_norm_from_diff_perf(integral_weights):
    return 1. / np.sum(integral_weights)

def get_global_mu_from_diff_perf(mus, integral_weights):
    return np.sum(mus * w / np.sum(integral_weights))

Calc the differential sensitivity, limits and discovery potential

In [ ]:
diff_mu_bfs, diff_cdfs, diff_pars, diff_flux_bfs = {}, {}, {}, {}

# Sensitity = avg. UL and discorvery potential
exp_sigma = prob2sigma(1. - bg_pdf.sf(result["ts"][tw_id]))[0]
sigmas = {"sens": 0., "lim": exp_sigma, "disc":  5.}
betas = {"sens": 0.9, "lim": 0.9, "disc":  0.5}

# Override seed [(df, loc), fscale] or [(df, loc, scale), None] per tw id per key
override_seed = {"lim": {0: [(1, 1), 0.4]}}  

# Get the BG TS values for the desired betas
diff_mu_bfs, diff_cdfs, diff_pars, diff_flux_bfs = {}, {}, {}, {}
for key in sigmas.keys():
    print(key)
    seed_override_dict = override_seed.get(key, None)
    
    bg_ts_val = bg_pdf.ppf(q=100. * sigma2prob(sigmas[key]))
    diff_mu_bfs[key], diff_cdfs[key], diff_pars[key] = fit_all_chi2_params(
        diff_trials["mus"], diff_trials["ts"], bg_ts_val, betas[key],
        seed_override_dict)
    diff_flux_bfs[key] = multi_sig_inj.mu2flux(np.array(diff_mu_bfs[key]))

Make the weight PDFs (inverse diff. Limits) and 90% intervals

In [ ]:
# Injection gamma for differential trials
gamma_diff_inj = 2.

bins = diff_trials["log_E_bins"]
diff_fluxes = diff_flux_bfs["lim"]  # From the diff. trials at 1GeV

int_weights, int_ws_norm = [], []
lo_ints, hi_ints = [], []
lo_idxs, hi_idxs = [], []
lo_ms, hi_ms = [], []
for i, gi in enumerate(gammas):
    # Get combination weigths
    pli_orig = power_law_integral(
        E0=1., gamma=gamma_diff_inj, logE_bins=bins, phi0=1.)
    # phi0 = 1 to have the c0 scaling directly comparable to phi0 = diff_fluxes
    pli_new = power_law_integral(E0=1., gamma=gi, logE_bins=bins, phi0=1)
    w = pli_new / (diff_fluxes * pli_orig)
    int_weights.append(w)
    
    # Norm to PDF
    dE = np.diff(bins) 
    w = w / np.sum(w) / dE
    int_ws_norm.append(w)

    # Mark 90% central interval in weight PDF
    w_cdf = np.cumsum(w * dE)
    lo_ms.append(w_cdf > 0.05)
    hi_ms.append(w_cdf < 0.95)
    lo_idxs.append(np.where(lo_ms[-1])[0][0])
    hi_idxs.append(np.where(hi_ms[-1])[0][-1])
    lo_ints.append(bins[lo_idxs[-1]])
    hi_ints.append(bins[hi_idxs[-1] + 1])

### Plot logE dependent limits for selected gammas

Note: Because the power law is fixed at E^-2, the E2 weighting, does not need to be explicitely done.
The flux norm at 1GeV is the same numerical value as the on at 100TeV weighted with 100TeV^2 (E^-2 is flat in E2 weighting).

In [ ]:
# Select a subset of gammas to plot
gammas_sub = [1.5, 2., 2.5]
sub_idx = np.where([gammas == g for g in gammas_sub])[1]
print("Plotting gammas: ", sub_idx)

Plot the weight PDFs per gamma

In [ ]:
c = ["0.0", "0.5", "0.35"]

fs = latex_setup.make_figsize()
fig, ax = plt.subplots(1, 1, figsize=fs)
ylim_min = 1e-2

for j, gamma_i in enumerate(gammas_sub):
    i = sub_idx[j]
   
    # Truncate manually for log plot for pgf out
    yvals = np.r_[int_ws_norm[i][0], int_ws_norm[i]]
    mask = (yvals < ylim_min)
    yvals[mask] = ylim_min // 10.
    
    ax.vlines(lo_ints[i], ylim_min, yvals[1:][lo_idxs[i]],
              linestyles="--", colors=c[j], zorder=-1)
    ax.vlines(hi_ints[i], ylim_min, yvals[1:][hi_idxs[i]],
              linestyles="--", color=c[j], zorder=-1)
    
    ax.plot(bins, yvals, drawstyle="steps-pre",
            color=c[j], label=r"$\gamma={:.1f}$".format(gamma_i))
    sl = slice(lo_idxs[i], hi_idxs[i] + 2, 1)
    ax.fill_between(bins[sl], ylim_min, yvals[sl],
                    color="C7", step="pre", alpha=.25)
    
ax.set_xlabel(r"$\log_{10}(E_\nu\, /\, \mathrm{GeV})$")
ax.set_ylabel("PDF")
ax.set_title(r"Injected $\gamma={:.0f}$".format(gamma_diff_inj))
ax.set_xlim(2, 9)
ax.set_yscale("log")
ax.set_ylim(ylim_min, 5e0)
# ax.set_ylim(0, None)
ax.legend(loc="upper center", ncol=3)

fig.tight_layout()
save_fig(fig, "plots/discussion_tdep/diff_weights.pdf", bbox_inches="tight")
plt.show()

Plot global upper limits from global trials and proper ranges from the differential trials

In [ ]:
def power_law(E, phi0, E0, gamma):
    return phi0 * (E / E0)**-gamma

gamma_weight = 2.  # E2 weighted fluxes

fs = latex_setup.make_figsize()
fig, ax = plt.subplots(1, 1, figsize=fs)

for j, gamma_i in enumerate(gammas_sub):
    i = sub_idx[j]
    
    # Here we can directly get the gloval limit from the direct trials
    # c0 = get_global_norm_from_diff_perf(integral_weights=int_weights[i])
    c0 = flux_bfs[i]
    
    x = np.logspace(lo_ints[i], hi_ints[i], 100)
    y = weight_Egamma(
        x, power_law(x, c0, E0=1., gamma=gamma_i), gamma=gamma_weight)
    
    x_dash = np.logspace(2, 9, 100)
    y_dash = weight_Egamma(
        x_dash, power_law(x_dash, c0, E0=1., gamma=gamma_i), gamma=gamma_weight)
    
    xerr = np.logspace(np.log10(x[0]), np.log10(x[-1]), 5)
    yerr = weight_Egamma(
        xerr, power_law(xerr, c0, E0=1., gamma=gamma_i), gamma=gamma_weight)
    
    ax.plot(x_dash, y_dash, c=c[j], ls="--", lw=1.5, zorder=-1)
    ax.plot(x, y, c=c[j], label=r"$\gamma={:.1f}$".format(gamma_i), lw=2.5)
    ax.errorbar(xerr, yerr, yerr=yerr / 5., uplims=True, fmt=",", color=c[j])
       
ax.set_xscale("log")
ax.set_yscale("log")
ax.set_xlabel(r"$E_\nu$ in $\mathrm{GeV}$")
plt.title("Time window {}".format(tw_id + 1))
ax.set_ylabel(r"$E^2\Phi$ in $\si[per-mode=reciprocal]{\GeV\per\cm\squared}$")
ax.set_xlim(1e2, 1e9)
ax.set_ylim(5e-2, 5e1)  # Manual limits
ax.legend(ncol=3, loc="upper center")

fig.tight_layout()
save_fig(fig, "plots/discussion_tdep/limits_gamma_logE.pdf",
         bbox_inches="tight")
plt.show()

### Plot gamma vs flux norm

In [ ]:
dgamma = np.diff(mplt.get_bins_from_mids(gammas)) / 2.

fs = latex_setup.make_figsize(ratio=2.5)
fig, axs = plt.subplots(1, 2, figsize=fs)

# At 1 GeV on the left, at 100 TeV on the right
ylabels = [r"$E^2\Phi_0$ at $\SI{1}{\GeV}$", r"$E^2\Phi_0$ at $\SI{100}{\TeV}$"]
yerr_scale = (1.5, 3.)
for i, E0new in enumerate([1e0, 1e5]):
    ax = axs[i]
    fluxes = np.array([weight_Egamma(
        E0new, fluxatE0(flx, E0=1., E0new=E0new, gamma=g), gamma=2.)
                       for flx, g in zip(flux_bfs, gammas)])

    ax.axhline(fluxes[gammas == 2.], 0, 1, ls="-", c="0.6")
    ax.axvline(2., 0, 1, ls="-", c="0.6")

    ax.plot(gammas, fluxes, c="#353132")
    sl = slice(1, -1, 2)
    ax.errorbar(gammas[sl], fluxes[sl], uplims=True,
                yerr=fluxes[sl] / yerr_scale[i],
                xerr=[-0.8*dgamma[sl], 0.8*dgamma[sl]],
                lw=2, c="#353132", fmt=",", ls="",
                label="Exp. 90\% UL", zorder=10)

    ax.set_ylim(round_to_decade(np.amin(fluxes), "floor"),
                round_to_decade(np.amax(fluxes), "ceil"))
    ax.set_ylabel(ylabels[i])

# Common
for ax in axs:
    ax.set_xlabel(r"$\gamma$")
    ax.set_yscale("log")
    ax.set_xlim(gammas[0] - dgamma[0], gammas[-1] + dgamma[-1])
    ax.set_xticks(np.arange(gammas[0], gammas[-1] + 0.5, 0.5))    
    ax.grid(ls=":")

fig.tight_layout()
save_fig(fig, "plots/discussion_tdep/limits_norm_vs_gammas.pdf",
         bbox_inches="tight")
plt.show()